In [1]:
# Might need to install arm - if so, uncomment the following line
#install.packages("arm")
library(arm)
library(lme4)
library(blme)
library(ggplot2)
library(stringr)
library(psycho)
library(glmnet)
library(car)
library("multcomp")

Loading required package: MASS

Loading required package: Matrix

Loading required package: lme4


arm (Version 1.13-1, built: 2022-8-25)


Working directory is /Users/tommccoy/Documents/tommccoy/Documents/GitHub/embers-of-autoregression/evaluation



Attaching package: ‘psycho’


The following object is masked from ‘package:lme4’:

    golden


Loaded glmnet 4.1-7

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:arm’:

    logit


Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data


Attaching package: ‘TH.data’


The following object is masked from ‘package:MASS’:

    geyser




In [2]:
# Set ggplot theme
base_size <- 20
theme_set(theme_bw(base_size=base_size) +
            theme(#panel.grid.major=element_blank(),
              panel.grid.minor=element_blank(),
              axis.title.y=element_text(angle=90,vjust=0.5),
              axis.text.x=element_text(angle=0, hjust=0.5),
              axis.title.x=element_blank()))

In [3]:
correct_vs_length_and_prob <- function(df, include_output_chars=TRUE, include_input_chars=TRUE){
 
    
    if (include_output_chars & include_input_chars){
        model <- glm(correct ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars + output_nchars, 
               data=df, family=binomial)
    } else if (include_output_chars & !include_input_chars){
        model <- glm(correct ~ input_logprob + output_logprob + input_ntokens + output_ntokens + output_nchars, 
               data=df, family=binomial)
    } else if (!include_output_chars & include_input_chars){
        model <- glm(correct ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars, 
               data=df, family=binomial)
    }
  
    
  return(model)
}

In [4]:
# For Z-scoring datasets

scale_df <- function(df) {
    new_df <- data.frame(scale(df[2:7]))
    new_df$index <- as.factor(df$index)
    new_df$correct <- df$correct
    
    return(new_df)
}

scale_taskpair_df <- function(df) {
    new_df <- data.frame(scale(df[3:8]))
    new_df$index <- as.factor(df$index)
    new_df$task <- as.factor(df$task)
    new_df$correct <- df$correct
    
    return(new_df)
}

scale_taskpair_prob_df <- function(df) {
    new_df <- data.frame(scale(df[2:8]))
    new_df$index <- as.factor(df$index)
    new_df$correct <- df$correct
    
    return(new_df)
}



scale_df_with_index <- function(df) {
    new_df <- data.frame(scale(df[1:7]))
    new_df$correct <- df$correct
    
    return(new_df)
}

# Shift ciphers

In [6]:
# Read in data
rot13enc_gpt4_df <- read.table(file = 'table_rot13enc_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13enc_gpt35_df <- read.table(file = 'table_rot13enc_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

rot13dec_gpt4_df <- read.table(file = 'table_rot13dec_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13dec_gpt35_df <- read.table(file = 'table_rot13dec_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [7]:
# Z-score data
scaled_rot13enc_gpt4_df <- scale_df(rot13enc_gpt4_df)
scaled_rot13enc_gpt35_df <- scale_df(rot13enc_gpt35_df)

scaled_rot13dec_gpt4_df <- scale_df(rot13dec_gpt4_df)
scaled_rot13dec_gpt35_df <- scale_df(rot13dec_gpt35_df)


In [8]:

rot13enc_gpt4_model <- correct_vs_length_and_prob(scaled_rot13enc_gpt4_df, include_output_chars=FALSE)



In [9]:
rot13enc_gpt35_model <- correct_vs_length_and_prob(scaled_rot13enc_gpt35_df, include_output_chars=FALSE)


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [10]:
rot13dec_gpt4_model <- correct_vs_length_and_prob(scaled_rot13dec_gpt4_df, include_output_chars=FALSE)


In [11]:
rot13dec_gpt35_model <- correct_vs_length_and_prob(scaled_rot13dec_gpt35_df, include_output_chars=FALSE)

In [12]:
summary(rot13enc_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.99812  -0.44168  -0.14960  -0.01557   2.95372  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -4.1263     0.5728  -7.204 5.84e-13 ***
input_logprob    1.2287     0.4840   2.539  0.01112 *  
output_logprob   7.8371     3.2790   2.390  0.01684 *  
input_ntokens    2.7704     0.9766   2.837  0.00456 ** 
output_ntokens  -0.9145     2.5316  -0.361  0.71793    
input_nchars     3.5012     3.3936   1.032  0.30221    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 242.98  on 299  degrees of freedom
Residual deviance: 155.44  on 294  degrees of freedom
AIC: 167.44

Number of Fisher Scoring iterations: 7


In [13]:
summary(rot13enc_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5360  -0.0774  -0.0159  -0.0002   3.1706  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -11.2249     2.6091  -4.302 1.69e-05 ***
input_logprob   -0.5781     1.1487  -0.503   0.6147    
output_logprob  20.4597    10.2690   1.992   0.0463 *  
input_ntokens    4.4254     2.7395   1.615   0.1062    
output_ntokens   6.4896     7.5227   0.863   0.3883    
input_nchars     1.4265     8.9552   0.159   0.8734    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 87.687  on 299  degrees of freedom
Residual deviance: 37.361  on 294  degrees of freedom
AIC: 49.361

Number of Fisher Scoring iterations: 10


In [14]:
summary(rot13dec_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5419  -0.8313  -0.4035   0.9128   2.6598  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -1.4549     0.2014  -7.224 5.04e-13 ***
input_logprob    0.8315     1.7469   0.476    0.634    
output_logprob   1.9277     0.3396   5.676 1.38e-08 ***
input_ntokens   -2.5684     1.6014  -1.604    0.109    
output_ntokens   0.8680     0.6047   1.436    0.151    
input_nchars     2.8939     2.0645   1.402    0.161    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 359.48  on 299  degrees of freedom
Residual deviance: 286.22  on 294  degrees of freedom
AIC: 298.22

Number of Fisher Scoring iterations: 6


In [15]:
summary(rot13dec_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4202  -0.3446  -0.1218  -0.0168   2.7923  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -4.8617     0.7477  -6.502 7.93e-11 ***
input_logprob    2.5098     3.4435   0.729   0.4661    
output_logprob   3.5218     0.8867   3.972 7.13e-05 ***
input_ntokens   -5.1178     2.8539  -1.793   0.0729 .  
output_ntokens   2.8505     1.0732   2.656   0.0079 ** 
input_nchars     4.3632     3.6236   1.204   0.2285    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 195.05  on 299  degrees of freedom
Residual deviance: 125.64  on 294  degrees of freedom
AIC: 137.64

Number of Fisher Scoring iterations: 8


In [16]:
vif(rot13enc_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.173461      42.710177       8.542567      30.461877      54.802060

In [17]:
vif(rot13enc_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.316592      39.463928       6.346975      26.275845      41.688708

In [18]:
vif(rot13dec_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
    104.840741       1.542007      88.685607      14.179527     149.580941

In [19]:
vif(rot13dec_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     61.877837       1.408309      44.281498       9.835931      75.711261

### Distance as output

In [20]:
# Read in data
rot13encdist_gpt4_df <- read.table(file = 'table_rot13enc_gpt-4-0613_dist.tsv', sep = '\t', header = TRUE)
rot13encdist_gpt35_df <- read.table(file = 'table_rot13enc_gpt-3.5-turbo-0613_dist.tsv', sep = '\t', header = TRUE)

In [21]:
# Get rid of "correct"
rot13encdist_gpt4_df <- rot13encdist_gpt4_df[,-c(1,8)]
rot13encdist_gpt35_df <- rot13encdist_gpt35_df[,-c(1,8)]

# Scale
rot13encdist_gpt4_df <- data.frame(scale(rot13encdist_gpt4_df))
rot13encdist_gpt35_df <- data.frame(scale(rot13encdist_gpt35_df))

In [22]:
model_dist4 <- glm(distance ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars, 
               data=rot13encdist_gpt4_df)
model_dist35 <- glm(distance ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars, 
               data=rot13encdist_gpt35_df)

In [23]:
summary(model_dist4)


Call:
glm(formula = distance ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, data = rot13encdist_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.33914  -0.34787  -0.03202   0.29298   2.81865  

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -2.046e-16  3.254e-02   0.000  1.00000    
input_logprob  -1.339e-01  4.522e-02  -2.960  0.00333 ** 
output_logprob -1.418e-01  3.300e-01  -0.430  0.66777    
input_ntokens  -1.138e+00  1.219e-01  -9.337  < 2e-16 ***
output_ntokens  1.029e+00  3.292e-01   3.126  0.00195 ** 
input_nchars    5.795e-01  4.344e-01   1.334  0.18315    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.3177005)

    Null deviance: 299.000  on 299  degrees of freedom
Residual deviance:  93.404  on 294  degrees of freedom
AIC: 515.31

Number of Fisher Scoring iterations: 2


In [24]:
summary(model_dist35)


Call:
glm(formula = distance ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, data = rot13encdist_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1784  -0.2853  -0.0955   0.0689  10.0305  

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)   
(Intercept)    -9.139e-16  5.090e-02   0.000  1.00000   
input_logprob  -2.170e-01  7.074e-02  -3.067  0.00236 **
output_logprob  9.036e-01  5.162e-01   1.751  0.08107 . 
input_ntokens  -5.331e-01  1.906e-01  -2.797  0.00550 **
output_ntokens  8.194e-01  5.150e-01   1.591  0.11267   
input_nchars    8.817e-01  6.794e-01   1.298  0.19540   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 0.7773369)

    Null deviance: 299.00  on 299  degrees of freedom
Residual deviance: 228.54  on 294  degrees of freedom
AIC: 783.74

Number of Fisher Scoring iterations: 2


In [25]:
vif(model_dist4)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.924853     102.499746      13.974510     102.010821     177.565292

In [26]:
vif(model_dist35)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.924853     102.499746      13.974510     102.010821     177.565292

### Comparing rot-2 to rot-13

In [27]:
# Read in data
rot13and2enc_gpt4_df <- read.table(file = 'table_rot13and2enc_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13and2enc_gpt35_df <- read.table(file = 'table_rot13and2enc_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

rot13and2dec_gpt4_df <- read.table(file = 'table_rot13and2dec_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13and2dec_gpt35_df <- read.table(file = 'table_rot13and2dec_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [28]:
# Z-score data
scaled_rot13and2enc_gpt4_df <- scale_taskpair_df(rot13and2enc_gpt4_df)
scaled_rot13and2enc_gpt35_df <- scale_taskpair_df(rot13and2enc_gpt35_df)

scaled_rot13and2dec_gpt4_df <- scale_taskpair_df(rot13and2dec_gpt4_df)
scaled_rot13and2dec_gpt35_df <- scale_taskpair_df(rot13and2dec_gpt35_df)


In [34]:
model_taskenc4 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2enc_gpt4_df, family=binomial)


In [35]:
model_taskenc35 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2enc_gpt35_df, family=binomial)

In [36]:
model_taskdec4 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2dec_gpt4_df, family=binomial)

In [37]:
model_taskdec35 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2dec_gpt35_df, family=binomial)

In [38]:
summary(model_taskenc4)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2enc_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.27805  -0.22008  -0.07750  -0.01265   2.79444  

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -2.874195   0.601776  -4.776 1.79e-06 ***
taskrot2enc_highprob -4.321099   1.546874  -2.793  0.00522 ** 
input_nchars         -0.005755   1.059544  -0.005  0.99567    
input_ntokens         2.000505   0.883266   2.265  0.02352 *  
output_ntokens       -1.134385   1.281776  -0.885  0.37615    
input_logprob         0.889370   0.735783   1.209  0.22676    
output_logprob        2.584170   1.456150   1.775  0.07595 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 134.373  on 199  degrees

In [39]:
summary(model_taskenc35)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2enc_gpt35_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.43256  -0.15092  -0.08247  -0.03489   2.66220  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -5.2207     1.4545  -3.589 0.000332 ***
taskrot2enc_highprob  -1.5693     1.4745  -1.064 0.287186    
input_nchars          -0.5423     1.1370  -0.477 0.633374    
input_ntokens          0.2442     1.0284   0.237 0.812345    
output_ntokens        -0.3968     1.1269  -0.352 0.724741    
input_logprob          0.8289     1.1217   0.739 0.459945    
output_logprob         0.5643     1.1385   0.496 0.620125    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 22.401  on 199  degrees of free

In [40]:
summary(model_taskdec4)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2dec_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4359  -0.4123  -0.2432   0.7928   2.6442  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -0.1211     0.2446  -0.495   0.6205    
taskrot2dec_highprob  -3.3529     0.6169  -5.435 5.49e-08 ***
input_nchars           0.3561     0.8967   0.397   0.6913    
input_ntokens         -0.7574     0.9265  -0.818   0.4136    
output_ntokens         0.9942     0.5949   1.671   0.0947 .  
input_logprob          0.1798     0.8221   0.219   0.8268    
output_logprob         1.0907     0.5100   2.139   0.0325 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 233.30  on 199  degrees of freedom
Residua

In [41]:
summary(model_taskdec35)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2dec_gpt35_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.28128  -0.30361  -0.09751  -0.02439   2.29539  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -2.3690     0.4916  -4.819 1.44e-06 ***
taskrot2dec_highprob  -4.3845     1.5528  -2.824  0.00475 ** 
input_nchars          -0.1466     0.9843  -0.149  0.88156    
input_ntokens         -0.8970     1.1125  -0.806  0.42011    
output_ntokens         2.1699     0.8523   2.546  0.01090 *  
input_logprob          0.7821     1.0182   0.768  0.44244    
output_logprob         2.3148     0.8666   2.671  0.00756 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 134.373  on 199  degrees of fre

In [42]:
vif(model_taskenc4)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.013204       3.801538       3.674851       4.271913       2.049254 
output_logprob 
      5.342965

In [43]:
vif(model_taskenc35)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.006233       1.303492       1.230690       1.255807       1.207468 
output_logprob 
      1.293834

In [44]:
vif(model_taskdec4)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.120305      17.699255      15.716925       8.339619      14.035269 
output_logprob 
      5.433576

In [45]:
vif(model_taskdec35)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.012453       4.677648       4.799392       4.384012       4.606440 
output_logprob 
      2.784557

### Input and output logprob for different prompt styles

In [46]:
# Read in data
rot13encbasic_gpt4_df <- read.table(file = 'table_rot13enc_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13decbasic_gpt4_df <- read.table(file = 'table_rot13dec_gpt-4-0613.tsv', sep = '\t', header = TRUE)

rot13encstep_gpt4_df <- read.table(file = 'table_rot13encstep_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13decstep_gpt4_df <- read.table(file = 'table_rot13decstep_gpt-4-0613.tsv', sep = '\t', header = TRUE)

rot13enccot_gpt4_df <- read.table(file = 'table_rot13enccot_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13deccot_gpt4_df <- read.table(file = 'table_rot13deccot_gpt-4-0613.tsv', sep = '\t', header = TRUE)


In [47]:
# Z-score data
scaled_rot13encbasic_gpt4_df <- scale_df(rot13encbasic_gpt4_df)
scaled_rot13decbasic_gpt4_df <- scale_df(rot13decbasic_gpt4_df)

scaled_rot13encstep_gpt4_df <- scale_df(rot13encstep_gpt4_df)
scaled_rot13decstep_gpt4_df <- scale_df(rot13decstep_gpt4_df)

scaled_rot13enccot_gpt4_df <- scale_df(rot13enccot_gpt4_df)
scaled_rot13deccot_gpt4_df <- scale_df(rot13deccot_gpt4_df)


In [48]:
rot13encbasic_gpt4_model <- correct_vs_length_and_prob(scaled_rot13encbasic_gpt4_df, include_output_chars=FALSE)
rot13decbasic_gpt4_model <- correct_vs_length_and_prob(scaled_rot13decbasic_gpt4_df, include_output_chars=FALSE)

rot13encstep_gpt4_model <- correct_vs_length_and_prob(scaled_rot13encstep_gpt4_df, include_output_chars=FALSE)
rot13decstep_gpt4_model <- correct_vs_length_and_prob(scaled_rot13decstep_gpt4_df, include_output_chars=FALSE)

rot13enccot_gpt4_model <- correct_vs_length_and_prob(scaled_rot13enccot_gpt4_df, include_output_chars=FALSE)
rot13deccot_gpt4_model <- correct_vs_length_and_prob(scaled_rot13deccot_gpt4_df, include_output_chars=FALSE)



In [49]:
summary(rot13encbasic_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.99812  -0.44168  -0.14960  -0.01557   2.95372  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -4.1263     0.5728  -7.204 5.84e-13 ***
input_logprob    1.2287     0.4840   2.539  0.01112 *  
output_logprob   7.8371     3.2790   2.390  0.01684 *  
input_ntokens    2.7704     0.9766   2.837  0.00456 ** 
output_ntokens  -0.9145     2.5316  -0.361  0.71793    
input_nchars     3.5012     3.3936   1.032  0.30221    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 242.98  on 299  degrees of freedom
Residual deviance: 155.44  on 294  degrees of freedom
AIC: 167.44

Number of Fisher Scoring iterations: 7


In [50]:
summary(rot13decbasic_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5419  -0.8313  -0.4035   0.9128   2.6598  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -1.4549     0.2014  -7.224 5.04e-13 ***
input_logprob    0.8315     1.7469   0.476    0.634    
output_logprob   1.9277     0.3396   5.676 1.38e-08 ***
input_ntokens   -2.5684     1.6014  -1.604    0.109    
output_ntokens   0.8680     0.6047   1.436    0.151    
input_nchars     2.8939     2.0645   1.402    0.161    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 359.48  on 299  degrees of freedom
Residual deviance: 286.22  on 294  degrees of freedom
AIC: 298.22

Number of Fisher Scoring iterations: 6


In [51]:
summary(rot13encstep_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.73379  -0.51483  -0.20383  -0.03238   2.75493  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -3.5963     0.4842  -7.427 1.11e-13 ***
input_logprob    0.8294     0.4394   1.888   0.0591 .  
output_logprob   6.8981     3.0990   2.226   0.0260 *  
input_ntokens    2.1459     0.9113   2.355   0.0185 *  
output_ntokens   0.4738     2.4126   0.196   0.8443    
input_nchars     2.1449     3.1837   0.674   0.5005    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 235.60  on 299  degrees of freedom
Residual deviance: 166.53  on 294  degrees of freedom
AIC: 178.53

Number of Fisher Scoring iterations: 7


In [52]:
summary(rot13decstep_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6949  -0.8505  -0.4023   0.8993   2.9190  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -0.9967     0.1704  -5.849 4.95e-09 ***
input_logprob    1.1856     1.6961   0.699  0.48452    
output_logprob   1.5328     0.2829   5.418 6.03e-08 ***
input_ntokens   -1.9730     1.5372  -1.283  0.19933    
output_ntokens   1.5806     0.6061   2.608  0.00911 ** 
input_nchars     1.5266     1.9743   0.773  0.43938    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 390.89  on 299  degrees of freedom
Residual deviance: 307.37  on 294  degrees of freedom
AIC: 319.37

Number of Fisher Scoring iterations: 5


In [53]:
summary(rot13enccot_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.28774  -0.20523  -0.04222  -0.00230   2.98322  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -7.5672     1.3560  -5.581  2.4e-08 ***
input_logprob    1.8554     0.9072   2.045   0.0408 *  
output_logprob   7.4052     5.2162   1.420   0.1557    
input_ntokens    4.2542     1.8051   2.357   0.0184 *  
output_ntokens  -6.2785     4.9176  -1.277   0.2017    
input_nchars     5.2889     5.8233   0.908   0.3638    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 141.63  on 299  degrees of freedom
Residual deviance:  77.40  on 294  degrees of freedom
AIC: 89.4

Number of Fisher Scoring iterations: 9


In [54]:
summary(rot13deccot_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9243  -1.0111   0.6140   0.8735   2.4145  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     0.16840    0.13331   1.263    0.207    
input_logprob   0.83460    1.42938   0.584    0.559    
output_logprob  1.29187    0.21888   5.902 3.59e-09 ***
input_ntokens  -2.08775    1.36528  -1.529    0.126    
output_ntokens  0.08215    0.50933   0.161    0.872    
input_nchars    2.97586    1.83157   1.625    0.104    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 412.47  on 299  degrees of freedom
Residual deviance: 339.83  on 294  degrees of freedom
AIC: 351.83

Number of Fisher Scoring iterations: 4


In [55]:
vif(rot13encbasic_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.173461      42.710177       8.542567      30.461877      54.802060

In [56]:
vif(rot13decbasic_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
    104.840741       1.542007      88.685607      14.179527     149.580941

In [57]:
vif(rot13encstep_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.225533      45.311636       8.153285      32.436396      55.861871

In [58]:
vif(rot13decstep_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      95.96988        1.40042       79.25660       14.38488      133.12316

In [59]:
vif(rot13enccot_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.110543      32.923383       8.813669      32.137163      53.213976

In [60]:
vif(rot13deccot_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
    102.879864       1.518292      94.159210      13.476369     170.169706

### Input logprob for different prompt styles with distance as output

In [61]:
# Read in data
rot13encbasicdist_gpt4_df <- read.table(file = 'table_rot13enc_gpt-4-0613_dist.tsv', sep = '\t', header = TRUE)
rot13encstepdist_gpt4_df <- read.table(file = 'table_rot13encstep_gpt-4-0613_dist.tsv', sep = '\t', header = TRUE)
rot13enccotdist_gpt4_df <- read.table(file = 'table_rot13enccot_gpt-4-0613_dist.tsv', sep = '\t', header = TRUE)



In [62]:
# Get rid of "correct"
rot13encbasicdist_gpt4_df <- rot13encbasicdist_gpt4_df[,-c(1,8)]
rot13encstepdist_gpt4_df <- rot13encstepdist_gpt4_df[,-c(1,8)]
rot13enccotdist_gpt4_df <- rot13enccotdist_gpt4_df[,-c(1,8)]


# Scale
scaled_rot13encbasicdist_gpt4_df <- data.frame(scale(rot13encbasicdist_gpt4_df))
scaled_rot13encstepdist_gpt4_df <- data.frame(scale(rot13encstepdist_gpt4_df))
scaled_rot13enccotdist_gpt4_df <- data.frame(scale(rot13enccotdist_gpt4_df))



In [63]:
modelbasic_dist4 <- glm(distance ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars, 
               data=rot13encbasicdist_gpt4_df)
modelstep_dist4 <- glm(distance ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars, 
               data=rot13encstepdist_gpt4_df)
modelcot_dist4 <- glm(distance ~ input_logprob + output_logprob + input_ntokens + output_ntokens + input_nchars, 
               data=rot13enccotdist_gpt4_df)

In [64]:
summary(modelbasic_dist4)


Call:
glm(formula = distance ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, data = rot13encbasicdist_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-11.9233   -3.0973   -0.2851    2.6086   25.0964  

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -2.171012   0.977895  -2.220  0.02718 *  
input_logprob  -0.021256   0.007181  -2.960  0.00333 ** 
output_logprob -0.009503   0.022119  -0.430  0.66777    
input_ntokens  -1.236568   0.132434  -9.337  < 2e-16 ***
output_ntokens  0.427956   0.136915   3.126  0.00195 ** 
input_nchars    0.111892   0.083861   1.334  0.18315    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 25.18589)

    Null deviance: 23703.4  on 299  degrees of freedom
Residual deviance:  7404.7  on 294  degrees of freedom
AIC: 1827.2

Number of Fisher Scoring iterations: 2


In [65]:
summary(modelstep_dist4)


Call:
glm(formula = distance ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, data = rot13encstepdist_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-12.8705   -2.8071   -0.2109    2.5441   19.1410  

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -1.594581   0.964343  -1.654   0.0993 .  
input_logprob  -0.018180   0.007081  -2.567   0.0107 *  
output_logprob -0.002303   0.021813  -0.106   0.9160    
input_ntokens  -1.147949   0.130598  -8.790   <2e-16 ***
output_ntokens  0.312296   0.135018   2.313   0.0214 *  
input_nchars    0.164673   0.082699   1.991   0.0474 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 24.4927)

    Null deviance: 21311.9  on 299  degrees of freedom
Residual deviance:  7200.9  on 294  degrees of freedom
AIC: 1818.8

Number of Fisher Scoring iterations: 2


In [66]:
summary(modelcot_dist4)


Call:
glm(formula = distance ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, data = rot13enccotdist_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-14.799   -4.138   -1.006    3.792   21.962  

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -2.06029    1.24189  -1.659   0.0982 .  
input_logprob  -0.01683    0.00912  -1.846   0.0659 .  
output_logprob -0.04055    0.02809  -1.444   0.1499    
input_ntokens  -1.11568    0.16819  -6.634 1.57e-10 ***
output_ntokens  0.40731    0.17388   2.343   0.0198 *  
input_nchars    0.04840    0.10650   0.454   0.6498    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for gaussian family taken to be 40.62025)

    Null deviance: 34779  on 299  degrees of freedom
Residual deviance: 11942  on 294  degrees of freedom
AIC: 1970.6

Number of Fisher Scoring iterations: 2


In [67]:
vif(modelbasic_dist4)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.924853     102.499746      13.974510     102.010821     177.565292

In [68]:
vif(modelstep_dist4)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.924853     102.499746      13.974510     102.010821     177.565292

In [69]:
vif(modelcot_dist4)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.924853     102.499746      13.974510     102.010821     177.565292

### Comparing rot-13 and rot-2 for different prompt styles

In [70]:
# Read in data
rot13and2encbasic_gpt4_df <- read.table(file = 'table_rot13and2enc_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13and2encstep_gpt4_df <- read.table(file = 'table_rot13and2encstep_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13and2enccot_gpt4_df <- read.table(file = 'table_rot13and2enccot_gpt-4-0613.tsv', sep = '\t', header = TRUE)

rot13and2decbasic_gpt4_df <- read.table(file = 'table_rot13and2dec_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13and2decstep_gpt4_df <- read.table(file = 'table_rot13and2decstep_gpt-4-0613.tsv', sep = '\t', header = TRUE)
rot13and2deccot_gpt4_df <- read.table(file = 'table_rot13and2deccot_gpt-4-0613.tsv', sep = '\t', header = TRUE)


In [71]:
# Z-score data
scaled_rot13and2encbasic_gpt4_df <- scale_taskpair_df(rot13and2encbasic_gpt4_df)
scaled_rot13and2encstep_gpt4_df <- scale_taskpair_df(rot13and2encstep_gpt4_df)
scaled_rot13and2enccot_gpt4_df <- scale_taskpair_df(rot13and2enccot_gpt4_df)


scaled_rot13and2decbasic_gpt4_df <- scale_taskpair_df(rot13and2decbasic_gpt4_df)
scaled_rot13and2decstep_gpt4_df <- scale_taskpair_df(rot13and2decstep_gpt4_df)
scaled_rot13and2deccot_gpt4_df <- scale_taskpair_df(rot13and2deccot_gpt4_df)



In [72]:
model_taskencbasic4 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2encbasic_gpt4_df, family=binomial)
model_taskencstep4 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2encstep_gpt4_df, family=binomial)
model_taskenccot4 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2enccot_gpt4_df, family=binomial)

model_taskdecbasic4 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2decbasic_gpt4_df, family=binomial)
model_taskdecstep4 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2decstep_gpt4_df, family=binomial)
model_taskdeccot4 <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and2deccot_gpt4_df, family=binomial)

In [73]:
summary(model_taskencbasic4)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2encbasic_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.27805  -0.22008  -0.07750  -0.01265   2.79444  

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -2.874195   0.601776  -4.776 1.79e-06 ***
taskrot2enc_highprob -4.321099   1.546874  -2.793  0.00522 ** 
input_nchars         -0.005755   1.059544  -0.005  0.99567    
input_ntokens         2.000505   0.883266   2.265  0.02352 *  
output_ntokens       -1.134385   1.281776  -0.885  0.37615    
input_logprob         0.889370   0.735783   1.209  0.22676    
output_logprob        2.584170   1.456150   1.775  0.07595 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 134.373  on 199  de

In [74]:
summary(model_taskencstep4)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2encstep_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.17351  -0.27674  -0.09821  -0.02221   2.59970  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.46595    0.51133  -4.823 1.42e-06 ***
taskrot2encstep_highprob -4.20757    1.53336  -2.744  0.00607 ** 
input_nchars             -0.02859    1.02087  -0.028  0.97766    
input_ntokens             1.36581    0.80535   1.696  0.08990 .  
output_ntokens           -0.59194    1.08706  -0.545  0.58607    
input_logprob             0.79898    0.70636   1.131  0.25800    
output_logprob            1.99277    1.27315   1.565  0.11753    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null devian

In [75]:
summary(model_taskenccot4)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2enccot_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.09833  -0.21603  -0.05857  -0.00944   1.86068  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -4.62384    1.00878  -4.584 4.57e-06 ***
taskrot2enccot_highprob -3.45087    1.48371  -2.326   0.0200 *  
input_nchars            -0.05547    1.11507  -0.050   0.9603    
input_ntokens            2.04535    1.10870   1.845   0.0651 .  
output_ntokens          -0.69142    1.23346  -0.561   0.5751    
input_logprob            1.22494    0.97408   1.258   0.2086    
output_logprob           3.60140    1.80598   1.994   0.0461 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 85.19

In [76]:
summary(model_taskdecbasic4)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2decbasic_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4359  -0.4123  -0.2432   0.7928   2.6442  

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)           -0.1211     0.2446  -0.495   0.6205    
taskrot2dec_highprob  -3.3529     0.6169  -5.435 5.49e-08 ***
input_nchars           0.3561     0.8967   0.397   0.6913    
input_ntokens         -0.7574     0.9265  -0.818   0.4136    
output_ntokens         0.9942     0.5949   1.671   0.0947 .  
input_logprob          0.1798     0.8221   0.219   0.8268    
output_logprob         1.0907     0.5100   2.139   0.0325 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 233.30  on 199  degrees of freedom
Re

In [77]:
summary(model_taskdecstep4)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2decstep_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6640  -0.8047  -0.3517   0.7893   2.4030  

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -0.05560    0.25395  -0.219   0.8267    
taskrot2decstep_highprob -1.68691    0.39733  -4.246 2.18e-05 ***
input_nchars              0.04472    0.85979   0.052   0.9585    
input_ntokens            -0.64643    0.85972  -0.752   0.4521    
output_ntokens            0.39650    0.52454   0.756   0.4497    
input_logprob             0.11562    0.78154   0.148   0.8824    
output_logprob            0.95721    0.46558   2.056   0.0398 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 261.37

In [78]:
summary(model_taskdeccot4)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and2deccot_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1577  -0.8867   0.5736   0.7239   1.8240  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              1.41000    0.28275   4.987 6.14e-07 ***
taskrot2deccot_highprob -0.49165    0.40436  -1.216  0.22404    
input_nchars             0.62867    0.89690   0.701  0.48334    
input_ntokens            0.58590    0.77800   0.753  0.45140    
output_ntokens           0.05053    0.48148   0.105  0.91641    
input_logprob            0.84259    0.79638   1.058  0.29004    
output_logprob           1.10417    0.41744   2.645  0.00817 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 229.22  on 199 

In [79]:
vif(model_taskencbasic4)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.013204       3.801538       3.674851       4.271913       2.049254 
output_logprob 
      5.342965

In [80]:
vif(model_taskencstep4)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.009392       4.282513       3.499993       3.991625       2.232371 
output_logprob 
      5.367884

In [81]:
vif(model_taskenccot4)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.014645       2.236259       2.540264       2.177203       1.578746 
output_logprob 
      3.464968

In [82]:
vif(model_taskdecbasic4)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.120305      17.699255      15.716925       8.339619      14.035269 
output_logprob 
      5.433576

In [83]:
vif(model_taskdecstep4)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.306934      14.267772      12.204563       5.974205      11.382629 
output_logprob 
      4.168989

In [84]:
vif(model_taskdeccot4)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.427416      29.993096      22.499894       8.670571      23.211449 
output_logprob 
      6.139726

In [86]:
# Comparing rot-13 and rot-12
rot13and12deccot_gpt4_df <- read.table(file = 'table_rot13and12deccot_gpt-4-0613.tsv', sep = '\t', header = TRUE)

scaled_rot13and12deccot_gpt4_df <- scale_taskpair_df(rot13and12deccot_gpt4_df)

model_rot13and12deccot_gpt4_df <- bayesglm(correct ~ task + input_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_rot13and12deccot_gpt4_df, family=binomial)

summary(model_rot13and12deccot_gpt4_df)


Call:
bayesglm(formula = correct ~ task + input_nchars + input_ntokens + 
    output_ntokens + input_logprob + output_logprob, family = binomial, 
    data = scaled_rot13and12deccot_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.15101  -0.08213  -0.04706   0.53706   1.39744  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.39453    0.27887   5.001 5.72e-07 ***
taskshiftcot_12 -7.60737    1.91115  -3.981 6.88e-05 ***
input_nchars     0.46638    0.92162   0.506    0.613    
input_ntokens    0.08506    0.88289   0.096    0.923    
output_ntokens  -0.55992    0.60273  -0.929    0.353    
input_logprob   -0.03679    0.84415  -0.044    0.965    
output_logprob   0.73405    0.53124   1.382    0.167    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 267.499  on 199  degrees of freedom
Residual deviance:  94.683  on 1

In [87]:
vif(model_rot13and12deccot_gpt4_df)

task   input_nchars  input_ntokens output_ntokens  input_logprob 
      1.013689      15.753870      13.145906       6.619662      12.438819 
output_logprob 
      5.123642

### Comparing prompt techniques

In [ ]:
# Read in data
rot13encprompts_gpt4_df <- read.table(file = 'table_rot13enc_prompt_comparison.tsv', sep = '\t', header = TRUE)
rot13decprompts_gpt4_df <- read.table(file = 'table_rot13dec_prompt_comparison.tsv', sep = '\t', header = TRUE)

In [ ]:
head(rot13encprompts_gpt4_df)

In [ ]:
scale_prompt_df <- function(df) {
    new_df <- data.frame(scale(df[3:8]))
    new_df$index <- as.factor(df$index)
    new_df$correct <- df$correct
    new_df$prompt <- df$prompt
    
    return(new_df)
}

scaled_rot13encprompts_gpt4_df <- scale_prompt_df(rot13encprompts_gpt4_df)
scaled_rot13decprompts_gpt4_df <- scale_prompt_df(rot13decprompts_gpt4_df)


In [ ]:
head(scaled_rot13encprompts_gpt4_df)

In [ ]:
scaled_rot13encprompts_gpt4_df$prompt <- factor(scaled_rot13encprompts_gpt4_df$prompt)
scaled_rot13decprompts_gpt4_df$prompt <- factor(scaled_rot13decprompts_gpt4_df$prompt)

contrasts(scaled_rot13encprompts_gpt4_df$prompt) <- contr.sum(3)
contrasts(scaled_rot13decprompts_gpt4_df$prompt) <- contr.sum(3)


In [ ]:
rot13encprompts_gpt4_model <- glmer(correct ~ prompt + (1|index), 
               data=scaled_rot13encprompts_gpt4_df, family=binomial)
rot13decprompts_gpt4_model <- glmer(correct ~ prompt + (1|index), 
               data=scaled_rot13decprompts_gpt4_df, family=binomial)

In [ ]:
rot13encprompts_gpt4_null_model <- glmer(correct ~ (1|index), 
               data=scaled_rot13encprompts_gpt4_df, family=binomial)
rot13decprompts_gpt4_null_model <- glmer(correct ~ (1|index), 
               data=scaled_rot13decprompts_gpt4_df, family=binomial)

In [ ]:
anova(rot13encprompts_gpt4_model,rot13encprompts_gpt4_null_model,prompt="Chisq")

In [ ]:
anova(rot13decprompts_gpt4_model,rot13decprompts_gpt4_null_model,prompt="Chisq")

In [ ]:
gpt4enc_rot13_multcomp <- glht(rot13encprompts_gpt4_model, linfct=mcp(prompt="Tukey"))
gpt4dec_rot13_multcomp <- glht(rot13decprompts_gpt4_model, linfct=mcp(prompt="Tukey"))


In [ ]:
summary(gpt4enc_rot13_multcomp)

In [ ]:
summary(gpt4dec_rot13_multcomp)

# Reversal

In [89]:
# Read in data
revenc_gpt4_df <- read.table(file = 'table_revenc_gpt-4-0613.tsv', sep = '\t', header = TRUE)
revenc_gpt35_df <- read.table(file = 'table_revenc_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

revdec_gpt4_df <- read.table(file = 'table_revdec_gpt-4-0613.tsv', sep = '\t', header = TRUE)
revdec_gpt35_df <- read.table(file = 'table_revdec_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [90]:
# Z-score data
scaled_revenc_gpt4_df <- scale_df(revenc_gpt4_df)
scaled_revenc_gpt35_df <- scale_df(revenc_gpt35_df)

scaled_revdec_gpt4_df <- scale_df(revdec_gpt4_df)
scaled_revdec_gpt35_df <- scale_df(revdec_gpt35_df)

In [91]:

revenc_gpt4_model <- correct_vs_length_and_prob(scaled_revenc_gpt4_df, include_output_chars=FALSE)
revenc_gpt35_model <- correct_vs_length_and_prob(scaled_revenc_gpt35_df, include_output_chars=FALSE)

revdec_gpt4_model <- correct_vs_length_and_prob(scaled_revdec_gpt4_df, include_output_chars=FALSE)
revdec_gpt35_model <- correct_vs_length_and_prob(scaled_revdec_gpt35_df, include_output_chars=FALSE)


In [92]:
summary(revenc_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4293   0.3408   0.4777   0.5977   1.8622  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     1.65177    0.17275   9.562   <2e-16 ***
input_logprob  -0.05292    0.20547  -0.258    0.797    
output_logprob -0.46448    0.78690  -0.590    0.555    
input_ntokens   2.72928    1.88548   1.448    0.148    
output_ntokens -3.24191    2.00655  -1.616    0.106    
input_nchars   -0.89622    0.65648  -1.365    0.172    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 294.61  on 299  degrees of freedom
Residual deviance: 252.38  on 294  degrees of freedom
AIC: 264.38

Number of Fisher Scoring iterations: 4


In [93]:
summary(revenc_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5268  -0.9842  -0.4943   1.0049   2.1479  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.64275    0.14184  -4.532 5.85e-06 ***
input_logprob   0.27657    0.22757   1.215    0.224    
output_logprob -0.11149    0.86103  -0.129    0.897    
input_ntokens  -1.68717    1.72867  -0.976    0.329    
output_ntokens  0.01594    1.73855   0.009    0.993    
input_nchars    0.64733    0.68799   0.941    0.347    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 400.34  on 299  degrees of freedom
Residual deviance: 341.51  on 294  degrees of freedom
AIC: 353.51

Number of Fisher Scoring iterations: 5


In [96]:
summary(revdec_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8716   0.1950   0.3063   0.4420   1.9434  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      2.1370     0.2382   8.972  < 2e-16 ***
input_logprob   -2.9227     0.9709  -3.010  0.00261 ** 
output_logprob   2.2588     0.3278   6.891 5.55e-12 ***
input_ntokens   -0.1544     2.3128  -0.067  0.94677    
output_ntokens  -0.9864     2.2231  -0.444  0.65727    
input_nchars    -0.4750     0.7494  -0.634  0.52616    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 282.84  on 299  degrees of freedom
Residual deviance: 193.97  on 294  degrees of freedom
AIC: 205.97

Number of Fisher Scoring iterations: 6


In [97]:
summary(revdec_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars, family = binomial, data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9950  -0.9029   0.5507   0.8212   2.5696  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     0.03854    0.14148   0.272  0.78530    
input_logprob  -2.61145    0.89685  -2.912  0.00359 ** 
output_logprob  1.51108    0.25516   5.922 3.18e-09 ***
input_ntokens  -2.93531    1.82509  -1.608  0.10777    
output_ntokens  0.62214    1.76568   0.352  0.72457    
input_nchars   -0.36102    0.72336  -0.499  0.61772    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 413.97  on 299  degrees of freedom
Residual deviance: 321.55  on 294  degrees of freedom
AIC: 333.55

Number of Fisher Scoring iterations: 5


In [94]:
vif(revenc_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      2.013055      25.525399     145.583379     162.766744      17.494424

In [95]:
vif(revenc_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.823671      26.238360      96.460687      98.773576      16.188054

In [98]:
vif(revdec_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     24.904118       3.325799     139.564288     126.082515      15.193505

In [99]:
vif(revdec_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     34.461652       1.759196     135.204630     127.860576      20.984670

# Swap

In [139]:
# Read in data
swap_next_base_gpt4_df <- read.table(file = 'table_swap_next_base_gpt-4-0613.tsv', sep = '\t', header = TRUE)
swap_next_base_gpt35_df <- read.table(file = 'table_swap_next_base_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)



In [140]:
# Z-score data
scaled_swap_next_base_gpt4_df <- scale_df(swap_next_base_gpt4_df)
scaled_swap_next_base_gpt35_df <- scale_df(swap_next_base_gpt35_df)



In [141]:
swap_next_base_gpt4_model <- glm(correct ~ input_logprob + output_logprob + input_ntokens + input_nchars, 
                                 data=scaled_swap_next_base_gpt4_df, family=binomial)
swap_next_base_gpt35_model <- glm(correct ~ input_logprob + output_logprob + input_ntokens + input_nchars, 
                                 data=scaled_swap_next_base_gpt35_df, family=binomial)


In [142]:
vif(swap_next_base_gpt4_model)

input_logprob output_logprob  input_ntokens   input_nchars 
      18.13056       11.62989       21.14057       17.05375

In [143]:
vif(swap_next_base_gpt35_model)

input_logprob output_logprob  input_ntokens   input_nchars 
      16.31701       10.30039       18.53580       15.99737

### Rerunning with just output logprob

In [144]:
swap_next_base_gpt4_model <- glm(correct ~ output_logprob + input_ntokens + input_nchars, 
                                 data=scaled_swap_next_base_gpt4_df, family=binomial)
swap_next_base_gpt35_model <- glm(correct ~ output_logprob + input_ntokens + input_nchars, 
                                 data=scaled_swap_next_base_gpt35_df, family=binomial)


In [145]:
summary(swap_next_base_gpt4_model)


Call:
glm(formula = correct ~ output_logprob + input_ntokens + input_nchars, 
    family = binomial, data = scaled_swap_next_base_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.23047  -0.54883  -0.07368   0.65085   2.53538  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -0.6818     0.1924  -3.543 0.000396 ***
output_logprob   3.5219     0.4123   8.541  < 2e-16 ***
input_ntokens    2.2323     0.7371   3.029 0.002457 ** 
input_nchars    -0.5648     0.6971  -0.810 0.417840    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 413.97  on 299  degrees of freedom
Residual deviance: 245.89  on 296  degrees of freedom
AIC: 253.89

Number of Fisher Scoring iterations: 6


In [146]:
summary(swap_next_base_gpt35_model)


Call:
glm(formula = correct ~ output_logprob + input_ntokens + input_nchars, 
    family = binomial, data = scaled_swap_next_base_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6911  -0.9031  -0.3101   0.9510   2.2064  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -0.8507     0.1628  -5.224 1.75e-07 ***
output_logprob   1.9801     0.2885   6.864 6.69e-12 ***
input_ntokens    0.2248     0.5931   0.379    0.705    
input_nchars     0.4359     0.5867   0.743    0.457    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 397.45  on 299  degrees of freedom
Residual deviance: 316.07  on 296  degrees of freedom
AIC: 324.07

Number of Fisher Scoring iterations: 5


In [147]:
vif(swap_next_base_gpt4_model)

output_logprob  input_ntokens   input_nchars 
      2.219958      18.270921      16.873024

In [148]:
vif(swap_next_base_gpt35_model)

output_logprob  input_ntokens   input_nchars 
      1.688885      15.839155      16.040525

# Pig Latin

In [100]:
# Read in data
pigenc_gpt4_df <- read.table(file = 'table_pig_ay_enc_gpt-4-0613.tsv', sep = '\t', header = TRUE)
pigenc_gpt35_df <- read.table(file = 'table_pig_ay_enc_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

pigdec_gpt4_df <- read.table(file = 'table_pig_ay_dec_gpt-4-0613.tsv', sep = '\t', header = TRUE)
pigdec_gpt35_df <- read.table(file = 'table_pig_ay_dec_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [101]:
# Z-score data
scaled_pigenc_gpt4_df <- scale_df(pigenc_gpt4_df)
scaled_pigenc_gpt35_df <- scale_df(pigenc_gpt35_df)

scaled_pigdec_gpt4_df <- scale_df(pigdec_gpt4_df)
scaled_pigdec_gpt35_df <- scale_df(pigdec_gpt35_df)

In [102]:

pigenc_gpt4_model <- correct_vs_length_and_prob(scaled_pigenc_gpt4_df, include_output_chars=TRUE)
pigenc_gpt35_model <- correct_vs_length_and_prob(scaled_pigenc_gpt35_df, include_output_chars=TRUE)

pigdec_gpt4_model <- correct_vs_length_and_prob(scaled_pigdec_gpt4_df, include_output_chars=TRUE)
pigdec_gpt35_model <- correct_vs_length_and_prob(scaled_pigdec_gpt35_df, include_output_chars=TRUE)

In [103]:
summary(pigenc_gpt4_model) 


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars + output_nchars, family = binomial, 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7494  -1.0129  -0.6483   1.1876   2.0373  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.52418    0.12731  -4.117 3.83e-05 ***
input_logprob  -0.02844    0.18989  -0.150  0.88097    
output_logprob  1.93966    0.70569   2.749  0.00598 ** 
input_ntokens   1.89304    0.79457   2.382  0.01720 *  
output_ntokens  1.90174    0.99917   1.903  0.05700 .  
input_nchars    5.93802    2.95840   2.007  0.04473 *  
output_nchars  -8.30880    3.67071  -2.264  0.02360 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 400.34  on 299  degrees of freedom
Residual deviance: 368.70  on 293  degrees of freedom
AIC: 382.7

Number of 

In [132]:
summary(pigenc_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars + output_nchars, family = binomial, 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.3523  -0.8287  -0.6536   1.1613   2.2372  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -1.16906    0.14519  -8.052 8.15e-16 ***
input_logprob   0.08184    0.21390   0.383   0.7020    
output_logprob  0.91205    0.74120   1.230   0.2185    
input_ntokens   1.72350    0.86877   1.984   0.0473 *  
output_ntokens -0.86339    1.09634  -0.788   0.4310    
input_nchars   -1.56653    3.29903  -0.475   0.6349    
output_nchars   1.17719    4.02761   0.292   0.7701    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 341.72  on 299  degrees of freedom
Residual deviance: 321.69  on 293  degrees of freedom
AIC: 335.69

Number of

In [135]:
summary(pigdec_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars + output_nchars, family = binomial, 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8502  -0.8200   0.4322   0.7549   2.2193  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      0.5917     0.1518   3.898 9.71e-05 ***
input_logprob    0.9396     0.7882   1.192    0.233    
output_logprob   2.0122     0.2523   7.976 1.51e-15 ***
input_ntokens    0.8526     1.1394   0.748    0.454    
output_ntokens  -1.0679     0.9202  -1.161    0.246    
input_nchars     4.2380     4.2569   0.996    0.319    
output_nchars   -1.4817     3.4520  -0.429    0.668    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 403.81  on 299  degrees of freedom
Residual deviance: 297.55  on 293  degrees of freedom
AIC: 311.55

Number of

In [136]:
summary(pigdec_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob + input_ntokens + 
    output_ntokens + input_nchars + output_nchars, family = binomial, 
    data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6092  -0.6466  -0.1985   0.6533   3.3620  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -2.0072     0.2743  -7.317 2.54e-13 ***
input_logprob    1.4048     0.8884   1.581   0.1138    
output_logprob   3.1852     0.4590   6.940 3.93e-12 ***
input_ntokens    0.3708     1.2622   0.294   0.7689    
output_ntokens  -2.3121     1.0911  -2.119   0.0341 *  
input_nchars     4.7209     4.8613   0.971   0.3315    
output_nchars   -0.2726     3.9111  -0.070   0.9444    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 357.64  on 299  degrees of freedom
Residual deviance: 236.09  on 293  degrees of freedom
AIC: 250.09

Number of

In [133]:
vif(pigenc_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.983628      26.456314      35.444108      54.850209     476.276454 
 output_nchars 
    731.962743

In [134]:
vif(pigenc_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
      1.880259      23.183786      34.509126      51.770025     465.307172 
 output_nchars 
    700.189807

In [137]:
vif(pigdec_gpt4_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     26.736928       2.687614      52.706416      33.868847     731.134137 
 output_nchars 
    485.106578

In [138]:
vif(pigdec_gpt35_model)

input_logprob output_logprob  input_ntokens output_ntokens   input_nchars 
     24.924262       2.136799      51.388274      40.332526     781.249074 
 output_nchars 
    504.626583

### Comparing Pig Latin and Boar Etruscan

In [104]:
# Read in data
pigboarenc_gpt4_df <- read.table(file = 'table_pig_boar_enc_gpt-4-0613.tsv', sep = '\t', header = TRUE)
pigboarenc_gpt35_df <- read.table(file = 'table_pig_boar_enc_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

pigboardec_gpt4_df <- read.table(file = 'table_pig_boar_dec_gpt-4-0613.tsv', sep = '\t', header = TRUE)
pigboardec_gpt35_df <- read.table(file = 'table_pig_boar_dec_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [105]:
# Z-score data
scaled_pigboarenc_gpt4_df <- scale_taskpair_df(pigboarenc_gpt4_df)
scaled_pigboarenc_gpt35_df <- scale_taskpair_df(pigboarenc_gpt35_df)

scaled_pigboardec_gpt4_df <- scale_taskpair_df(pigboardec_gpt4_df)
scaled_pigboardec_gpt35_df <- scale_taskpair_df(pigboardec_gpt35_df)

In [106]:
model_pigtaskenc4 <- bayesglm(correct ~ task + input_nchars + output_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_pigboarenc_gpt4_df, family=binomial)


In [107]:
model_pigtaskenc35 <- bayesglm(correct ~ task + input_nchars + output_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_pigboarenc_gpt35_df, family=binomial)

In [108]:
model_pigtaskdec4 <- bayesglm(correct ~ task + input_nchars + output_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_pigboardec_gpt4_df, family=binomial)

In [109]:
model_pigtaskdec35 <- bayesglm(correct ~ task + input_nchars + output_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_pigboardec_gpt35_df, family=binomial)

In [110]:
summary(model_pigtaskenc4)


Call:
bayesglm(formula = correct ~ task + input_nchars + output_nchars + 
    input_ntokens + output_ntokens + input_logprob + output_logprob, 
    family = binomial, data = scaled_pigboarenc_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.2647  -0.7784  -0.5503   1.0503   2.3496  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.56003    0.23023  -2.432 0.014998 *  
taskuv         -1.32615    0.38979  -3.402 0.000668 ***
input_nchars    0.22833    0.77500   0.295 0.768286    
output_nchars   0.17641    0.87201   0.202 0.839682    
input_ntokens  -0.06654    0.61653  -0.108 0.914051    
output_ntokens  0.16345    0.72926   0.224 0.822658    
input_logprob  -0.05815    0.31163  -0.187 0.851964    
output_logprob  1.02052    0.70345   1.451 0.146853    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 229.22  on 199  degr

In [111]:
summary(model_pigtaskenc35)


Call:
bayesglm(formula = correct ~ task + input_nchars + output_nchars + 
    input_ntokens + output_ntokens + input_logprob + output_logprob, 
    family = binomial, data = scaled_pigboarenc_gpt35_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.37642  -0.50706  -0.09424  -0.03471   2.18253  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -1.2589     0.3004  -4.191 2.78e-05 ***
taskuv          -4.7283     1.5892  -2.975  0.00293 ** 
input_nchars    -0.1590     0.8572  -0.186  0.85283    
output_nchars   -0.3834     0.9585  -0.400  0.68912    
input_ntokens    1.6958     0.9195   1.844  0.06516 .  
output_ntokens  -0.9901     0.9850  -1.005  0.31484    
input_logprob    0.5392     0.4245   1.270  0.20403    
output_logprob   0.6935     0.8150   0.851  0.39483    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 169.08  o

In [112]:
summary(model_pigtaskdec4)


Call:
bayesglm(formula = correct ~ task + input_nchars + output_nchars + 
    input_ntokens + output_ntokens + input_logprob + output_logprob, 
    family = binomial, data = scaled_pigboardec_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2255  -0.7148   0.4806   0.7299   1.5880  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.7180     0.3041   5.649 1.61e-08 ***
taskuv          -0.9530     0.4023  -2.369  0.01783 *  
input_nchars     0.1932     1.0595   0.182  0.85528    
output_nchars    3.0459     1.1176   2.725  0.00642 ** 
input_ntokens   -0.8356     0.7813  -1.069  0.28487    
output_ntokens  -0.8758     0.6667  -1.314  0.18901    
input_logprob    0.3197     0.6747   0.474  0.63565    
output_logprob   1.3628     0.3321   4.104 4.06e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 233.3  on 199  degre

In [113]:
summary(model_pigtaskdec35)


Call:
bayesglm(formula = correct ~ task + input_nchars + output_nchars + 
    input_ntokens + output_ntokens + input_logprob + output_logprob, 
    family = binomial, data = scaled_pigboardec_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8910  -0.8879  -0.3267   0.8992   2.4220  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     0.05523    0.24760   0.223   0.8235    
taskuv         -0.63794    0.36335  -1.756   0.0791 .  
input_nchars    0.94939    1.13571   0.836   0.4032    
output_nchars   1.86490    1.04344   1.787   0.0739 .  
input_ntokens  -0.11739    0.71666  -0.164   0.8699    
output_ntokens -1.00693    0.63874  -1.576   0.1149    
input_logprob   0.62989    0.66958   0.941   0.3468    
output_logprob  1.90463    0.38320   4.970 6.68e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 276.28  on 199  deg

In [114]:
vif(model_pigtaskenc4)

task   input_nchars  output_nchars  input_ntokens output_ntokens 
      1.180588      18.256812      22.960500      11.547414      14.917764 
 input_logprob output_logprob 
      3.039306      14.211618

In [115]:
vif(model_pigtaskenc35)

task   input_nchars  output_nchars  input_ntokens output_ntokens 
      1.010115       9.057400      11.303950      11.009513      10.693821 
 input_logprob output_logprob 
      2.297823       7.959579

In [116]:
vif(model_pigtaskdec4)

task   input_nchars  output_nchars  input_ntokens output_ntokens 
      1.252570      36.086732      38.895505      20.904500      14.557983 
 input_logprob output_logprob 
     15.330770       3.646998

In [117]:
vif(model_pigtaskdec35)

task   input_nchars  output_nchars  input_ntokens output_ntokens 
      1.257700      44.067836      36.950123      17.287508      13.977888 
 input_logprob output_logprob 
     15.288089       4.092615

### Comparing Pig Latin variants

In [118]:
# Read in data
pigprobenc_gpt4_df <- read.table(file = 'table_pig_prob_enc_gpt-4-0613.tsv', sep = '\t', header = TRUE)
pigprobenc_gpt35_df <- read.table(file = 'table_pig_prob_enc_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

pigprobdec_gpt4_df <- read.table(file = 'table_pig_prob_dec_gpt-4-0613.tsv', sep = '\t', header = TRUE)
pigprobdec_gpt35_df <- read.table(file = 'table_pig_prob_dec_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [119]:
# Z-score data
scaled_pigprobenc_gpt4_df <- scale_taskpair_prob_df(pigprobenc_gpt4_df)
scaled_pigprobenc_gpt35_df <- scale_taskpair_prob_df(pigprobenc_gpt35_df)

scaled_pigprobdec_gpt4_df <- scale_taskpair_prob_df(pigprobdec_gpt4_df)
scaled_pigprobdec_gpt35_df <- scale_taskpair_prob_df(pigprobdec_gpt35_df)

In [120]:
model_pigprobenc4 <- glm(correct ~ task + input_nchars + output_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_pigprobenc_gpt4_df, family=binomial)


In [121]:
model_pigprobenc35 <- glm(correct ~ task + input_nchars + output_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_pigprobenc_gpt35_df, family=binomial)


In [122]:
model_pigprobdec4 <- glm(correct ~ task + input_nchars + output_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_pigprobdec_gpt4_df, family=binomial)

In [123]:
model_pigprobdec35 <- glm(correct ~ task + input_nchars + output_nchars + input_ntokens + output_ntokens + input_logprob + output_logprob, 
               data=scaled_pigprobdec_gpt35_df, family=binomial)

In [124]:
summary(model_pigprobenc4)


Call:
glm(formula = correct ~ task + input_nchars + output_nchars + 
    input_ntokens + output_ntokens + input_logprob + output_logprob, 
    family = binomial, data = scaled_pigprobenc_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6702  -0.7804  -0.4988   0.9214   2.5315  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -1.26069    0.12461 -10.117  < 2e-16 ***
task            0.74354    0.12438   5.978 2.26e-09 ***
input_nchars    5.28312    1.50249   3.516 0.000438 ***
output_nchars  -6.55400    1.90123  -3.447 0.000566 ***
input_ntokens   0.82896    0.60496   1.370 0.170602    
output_ntokens  1.82726    0.71174   2.567 0.010249 *  
input_logprob   0.01677    0.22750   0.074 0.941239    
output_logprob  1.90058    0.58417   3.253 0.001140 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 581.26  on 499  degrees o

In [125]:
summary(model_pigprobenc35)


Call:
glm(formula = correct ~ task + input_nchars + output_nchars + 
    input_ntokens + output_ntokens + input_logprob + output_logprob, 
    family = binomial, data = scaled_pigprobenc_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5192  -0.6000  -0.3587  -0.1589   2.6930  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -2.1511     0.1796 -11.979  < 2e-16 ***
task             0.7805     0.1537   5.079 3.79e-07 ***
input_nchars     4.3925     1.8793   2.337  0.01943 *  
output_nchars   -6.2121     2.3870  -2.602  0.00926 ** 
input_ntokens    2.1263     0.7775   2.735  0.00624 ** 
output_ntokens   1.7178     0.9240   1.859  0.06302 .  
input_logprob    0.4887     0.2990   1.634  0.10217    
output_logprob   2.4526     0.7408   3.311  0.00093 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 452.70  on 499  degrees 

In [126]:
summary(model_pigprobdec4)


Call:
glm(formula = correct ~ task + input_nchars + output_nchars + 
    input_ntokens + output_ntokens + input_logprob + output_logprob, 
    family = binomial, data = scaled_pigprobdec_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.6147   0.2434   0.4481   0.6058   2.1224  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.7257     0.1439  11.994  < 2e-16 ***
task             0.2060     0.1288   1.600  0.10955    
input_nchars    -3.2086     2.2790  -1.408  0.15917    
output_nchars    6.2809     1.9138   3.282  0.00103 ** 
input_ntokens   -0.2165     0.7466  -0.290  0.77181    
output_ntokens   0.4469     0.7409   0.603  0.54638    
input_logprob    1.5419     0.6486   2.377  0.01745 *  
output_logprob   1.6343     0.2681   6.096 1.09e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 505.90  on 499  degrees o

In [127]:
summary(model_pigprobdec35)


Call:
glm(formula = correct ~ task + input_nchars + output_nchars + 
    input_ntokens + output_ntokens + input_logprob + output_logprob, 
    family = binomial, data = scaled_pigprobdec_gpt35_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.44809  -0.93313   0.04602   0.97348   2.74208  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.05707    0.10349  -0.551 0.581348    
task            0.11682    0.10485   1.114 0.265176    
input_nchars    1.12060    1.64945   0.679 0.496899    
output_nchars   2.39500    1.37888   1.737 0.082402 .  
input_ntokens   0.65655    0.59733   1.099 0.271708    
output_ntokens -2.08243    0.55620  -3.744 0.000181 ***
input_logprob   0.99006    0.50218   1.972 0.048665 *  
output_logprob  1.59434    0.24255   6.573 4.92e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 693.15  on 499

In [128]:
vif(model_pigprobenc4)

task   input_nchars  output_nchars  input_ntokens output_ntokens 
      1.110584     158.104614     246.548269      25.678194      34.858337 
 input_logprob output_logprob 
      3.726655      22.983225

In [129]:
vif(model_pigprobenc35)

task   input_nchars  output_nchars  input_ntokens output_ntokens 
      1.120713     130.867247     207.526554      24.733180      32.169775 
 input_logprob output_logprob 
      3.261630      19.732940

In [130]:
vif(model_pigprobdec4)

task   input_nchars  output_nchars  input_ntokens output_ntokens 
      1.043172     295.487490     206.041983      33.334175      31.339362 
 input_logprob output_logprob 
     26.372965       4.595437

In [131]:
vif(model_pigprobdec35)

task   input_nchars  output_nchars  input_ntokens output_ntokens 
      1.056881     252.950717     174.982765      33.519722      29.100523 
 input_logprob output_logprob 
     23.720685       4.967613

# Acronyms

In [149]:
# Read in data
acronym_gpt4_outp_df <- read.table(file = 'table_acronym_varyoutp_gpt-4-0613.tsv', sep = '\t', header = TRUE)
acronym_gpt35_outp_df <- read.table(file = 'table_acronym_varyoutp_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

acronym_gpt4_inp_df <- read.table(file = 'table_acronym_varyinp_gpt-4-0613.tsv', sep = '\t', header = TRUE)
acronym_gpt35_inp_df <- read.table(file = 'table_acronym_varyinp_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)


In [150]:
# Z-score data
scaled_acronym_gpt4_outp_df <- scale_df(acronym_gpt4_outp_df)
scaled_acronym_gpt35_outp_df <- scale_df(acronym_gpt35_outp_df)

scaled_acronym_gpt4_inp_df <- scale_df(acronym_gpt4_inp_df)
scaled_acronym_gpt35_inp_df <- scale_df(acronym_gpt35_inp_df)

In [151]:
acronym_gpt4_outp_model <- glm(correct ~ input_logprob + output_logprob, 
               data=scaled_acronym_gpt4_outp_df, family=binomial)
acronym_gpt35_outp_model <- glm(correct ~ input_logprob + output_logprob, 
               data=scaled_acronym_gpt35_outp_df, family=binomial)
acronym_gpt4_inp_model <- glm(correct ~ input_logprob + output_logprob, 
               data=scaled_acronym_gpt4_inp_df, family=binomial)
acronym_gpt35_inp_model <- glm(correct ~ input_logprob + output_logprob, 
               data=scaled_acronym_gpt35_inp_df, family=binomial)

In [156]:
summary(acronym_gpt4_outp_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob, family = binomial, 
    data = scaled_acronym_gpt4_outp_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8290  -1.4607   0.7400   0.8019   1.0212  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     1.02684    0.03230  31.795  < 2e-16 ***
input_logprob   0.01891    0.03233   0.585    0.559    
output_logprob  0.20402    0.03182   6.412 1.44e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5788.3  on 4999  degrees of freedom
Residual deviance: 5746.4  on 4997  degrees of freedom
AIC: 5752.4

Number of Fisher Scoring iterations: 4


In [157]:
summary(acronym_gpt35_outp_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob, family = binomial, 
    data = scaled_acronym_gpt35_outp_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1969  -0.8597  -0.7189   1.3686   2.1452  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.95746    0.03234 -29.605   <2e-16 ***
input_logprob   0.04300    0.03167   1.357    0.175    
output_logprob  0.39609    0.03396  11.664   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5970.6  on 4999  degrees of freedom
Residual deviance: 5821.2  on 4997  degrees of freedom
AIC: 5827.2

Number of Fisher Scoring iterations: 4


In [152]:
summary(acronym_gpt4_inp_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob, family = binomial, 
    data = scaled_acronym_gpt4_inp_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8250  -1.4850   0.7384   0.7858   0.9456  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     1.07649    0.03261  33.008  < 2e-16 ***
input_logprob   0.05493    0.03259   1.685   0.0919 .  
output_logprob  0.15358    0.03244   4.735 2.19e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5681.9  on 4999  degrees of freedom
Residual deviance: 5656.6  on 4997  degrees of freedom
AIC: 5662.6

Number of Fisher Scoring iterations: 4


In [153]:
summary(acronym_gpt35_inp_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob, family = binomial, 
    data = scaled_acronym_gpt35_inp_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1457  -0.9082  -0.8348   1.4117   1.7552  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.72733    0.03035 -23.961  < 2e-16 ***
input_logprob   0.14452    0.03027   4.775 1.80e-06 ***
output_logprob  0.14814    0.03056   4.847 1.25e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6323.2  on 4999  degrees of freedom
Residual deviance: 6276.6  on 4997  degrees of freedom
AIC: 6282.6

Number of Fisher Scoring iterations: 4


In [158]:
vif(acronym_gpt4_outp_model)

input_logprob output_logprob 
      1.003013       1.003013

In [159]:
vif(acronym_gpt35_outp_model)

input_logprob output_logprob 
       1.00262        1.00262

In [154]:
vif(acronym_gpt4_inp_model)

input_logprob output_logprob 
      1.000001       1.000001

In [155]:
vif(acronym_gpt35_inp_model)

input_logprob output_logprob 
      1.000048       1.000048

### Vary task

In [160]:
# Read in data
acronym_gpt4_1and2_df <- read.table(file = 'table_acronym_varytask_gpt-4-0613.tsv', sep = '\t', header = TRUE)
acronym_gpt35_1and2_df <- read.table(file = 'table_acronym_varytask_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)


In [161]:
# Z-score data
scaled_acronym_gpt4_1and2_df <- scale_taskpair_df(acronym_gpt4_1and2_df)
scaled_acronym_gpt35_1and2_df <- scale_taskpair_df(acronym_gpt35_1and2_df)

In [162]:
acronym_gpt4_1and2_model <- bayesglm(correct ~ task + input_logprob + output_logprob, 
               data=scaled_acronym_gpt4_1and2_df, family=binomial)
acronym_gpt35_1and2_model <- bayesglm(correct ~ task + input_logprob + output_logprob, 
               data=scaled_acronym_gpt35_1and2_df, family=binomial)

In [163]:
summary(acronym_gpt4_1and2_model)


Call:
bayesglm(formula = correct ~ task + input_logprob + output_logprob, 
    family = binomial, data = scaled_acronym_gpt4_1and2_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9392  -0.2452  -0.2053   0.6980   2.8814  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     1.18066    0.07532  15.674  < 2e-16 ***
taskacronym2   -4.82454    0.21268 -22.684  < 2e-16 ***
input_logprob  -0.06966    0.06591  -1.057 0.290499    
output_logprob  0.26550    0.07002   3.792 0.000149 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2682.0  on 1999  degrees of freedom
Residual deviance: 1323.3  on 1996  degrees of freedom
AIC: 1331.3

Number of Fisher Scoring iterations: 7


In [164]:
summary(acronym_gpt35_1and2_model)


Call:
bayesglm(formula = correct ~ task + input_logprob + output_logprob, 
    family = binomial, data = scaled_acronym_gpt35_1and2_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.20491  -0.93012  -0.02338  -0.02057   1.57853  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -0.44672    0.06506  -6.866 6.58e-12 ***
taskacronym2   -7.80307    1.83266  -4.258 2.06e-05 ***
input_logprob   0.04556    0.06047   0.753   0.4512    
output_logprob  0.15608    0.06549   2.383   0.0172 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1976.4  on 1999  degrees of freedom
Residual deviance: 1332.3  on 1996  degrees of freedom
AIC: 1340.3

Number of Fisher Scoring iterations: 16


In [165]:
vif(acronym_gpt4_1and2_model)

task  input_logprob output_logprob 
      1.016252       1.004068       1.019017

In [166]:
vif(acronym_gpt35_1and2_model)

task  input_logprob output_logprob 
      1.000015       1.003099       1.003112

# Multiplication

In [266]:
# Read in data
mult_gpt4_df <- read.table(file = 'table_multiplication_gpt-4-0613.tsv', sep = '\t', header = TRUE)
mult_gpt35_df <- read.table(file = 'table_multiplication_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [267]:
mult_gpt4_df$method <- factor(mult_gpt4_df$method)
mult_gpt35_df$method <- factor(mult_gpt35_df$method)

In [268]:
head(mult_gpt4_df)

,index,method,correct
,<int>,<fct>,<int>
1,0,multiplication_number,0
2,1,multiplication_number,1
3,2,multiplication_number,0
4,3,multiplication_number,0
5,4,multiplication_number,1
6,5,multiplication_number,0


In [269]:
contrasts(mult_gpt4_df$method) <- contr.sum(4)
contrasts(mult_gpt35_df$method) <- contr.sum(4)

In [270]:
mult_gpt4_model <- glmer(correct ~ method + (1|index), 
               data=mult_gpt4_df, family=binomial)
mult_gpt4_null_model <- glmer(correct ~ (1|index), 
               data=mult_gpt4_df, family=binomial)   

In [271]:
mult_gpt35_model <- glmer(correct ~ method + (1|index), 
               data=mult_gpt35_df, family=binomial)
mult_gpt35_null_model <- glmer(correct ~ (1|index), 
               data=mult_gpt35_df, family=binomial)   

In [272]:
anova(mult_gpt4_model,mult_gpt4_null_model,test="Chisq")

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
mult_gpt4_null_model,2,431.4605,439.4434,-213.7303,427.4605,NA,NA,NA
mult_gpt4_model,5,393.3867,413.3440,-191.6933,383.3867,44.07383,3,1.455654e-09


In [273]:
anova(mult_gpt35_model,mult_gpt35_null_model,test="Chisq")

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
mult_gpt35_null_model,2,395.8202,403.8031,-195.9101,391.8202,NA,NA,NA
mult_gpt35_model,5,324.5254,344.4827,-157.2627,314.5254,77.29482,3,1.16727e-16


In [274]:
gpt4_mult_multcomp <- glht(mult_gpt4_model, linfct=mcp(method="Tukey"))


In [275]:
summary(gpt4_mult_multcomp)


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: glmer(formula = correct ~ method + (1 | index), data = mult_gpt4_df, 
    family = binomial)

Linear Hypotheses:
                                                             Estimate
multiplication_alternatingcaps - multiplication_allcaps == 0  -2.1294
multiplication_number - multiplication_allcaps == 0            1.0515
multiplication_word - multiplication_allcaps == 0              0.3911
multiplication_number - multiplication_alternatingcaps == 0    3.1809
multiplication_word - multiplication_alternatingcaps == 0      2.5205
multiplication_word - multiplication_number == 0              -0.6604
                                                             Std. Error z value
multiplication_alternatingcaps - multiplication_allcaps == 0     0.5432  -3.920
multiplication_number - multiplication_allcaps == 0              0.4521   2.326
multiplication_word - multiplication_allcaps == 0 

In [276]:
gpt35_mult_multcomp <- glht(mult_gpt35_model, linfct=mcp(method="Tukey"))


In [277]:
summary(gpt35_mult_multcomp)


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: glmer(formula = correct ~ method + (1 | index), data = mult_gpt35_df, 
    family = binomial)

Linear Hypotheses:
                                                             Estimate
multiplication_alternatingcaps - multiplication_allcaps == 0  -2.8676
multiplication_number - multiplication_allcaps == 0            1.8968
multiplication_word - multiplication_allcaps == 0              0.1711
multiplication_number - multiplication_alternatingcaps == 0    4.7644
multiplication_word - multiplication_alternatingcaps == 0      3.0387
multiplication_word - multiplication_number == 0              -1.7257
                                                             Std. Error z value
multiplication_alternatingcaps - multiplication_allcaps == 0     1.0538  -2.721
multiplication_number - multiplication_allcaps == 0              0.4042   4.693
multiplication_word - multiplication_allcaps == 0

# Linear function

In [215]:
# Read in data
linfwd_gpt4_df <- read.table(file = 'table_conversion_fwd_gpt-4-0613.tsv', sep = '\t', header = TRUE)
linfwd_gpt35_df <- read.table(file = 'table_conversion_fwd_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

linrev_gpt4_df <- read.table(file = 'table_conversion_rev_gpt-4-0613.tsv', sep = '\t', header = TRUE)
linrev_gpt35_df <- read.table(file = 'table_conversion_rev_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [216]:
head(linfwd_gpt4_df)

,index,task,input,output,correct
,<int>,<chr>,<int>,<dbl>,<int>
1,0,conversion_actual,328,622.4,1
2,1,conversion_actual,941,1725.8,0
3,2,conversion_actual,476,888.8,0
4,3,conversion_actual,230,446.0,1
5,4,conversion_actual,577,1070.6,0
6,5,conversion_actual,64,147.2,1


In [217]:
scale_lin_df <- function(df) {
    new_df <- data.frame(scale(df[3:5]))
    new_df$index <- as.factor(df$index)
    new_df$correct <- df$correct
    new_df$task <- factor(df$task)
    
    return(new_df)
}

In [218]:
scaled_linfwd_gpt4_df <- scale_lin_df(linfwd_gpt4_df)
scaled_linfwd_gpt35_df <- scale_lin_df(linfwd_gpt35_df)
scaled_linrev_gpt4_df <- scale_lin_df(linrev_gpt4_df)
scaled_linrev_gpt35_df <- scale_lin_df(linrev_gpt35_df)

In [220]:
linfwd_gpt4_model <- bayesglm(correct ~ task + input + output, 
               data=scaled_linfwd_gpt4_df, family=binomial)
linfwd_gpt35_model <- bayesglm(correct ~ task + input + output, 
               data=scaled_linfwd_gpt35_df, family=binomial)
linrev_gpt4_model <- bayesglm(correct ~ task + input + output, 
               data=scaled_linrev_gpt4_df, family=binomial)
linrev_gpt35_model <- bayesglm(correct ~ task + input + output, 
               data=scaled_linrev_gpt35_df, family=binomial)

In [221]:
summary(linfwd_gpt4_model)


Call:
bayesglm(formula = correct ~ task + input + output, family = binomial, 
    data = scaled_linfwd_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.38997  -0.40293  -0.08789  -0.03170   2.28552  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)   
(Intercept)          -0.8416     0.2924  -2.878  0.00400 **
taskconversion_fake  -5.4764     1.6658  -3.288  0.00101 **
input                -0.5816     0.8363  -0.695  0.48679   
output               -0.5344     0.7688  -0.695  0.48703   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 179.15  on 199  degrees of freedom
Residual deviance: 104.27  on 196  degrees of freedom
AIC: 112.27

Number of Fisher Scoring iterations: 18


In [222]:
summary(linfwd_gpt35_model)


Call:
bayesglm(formula = correct ~ task + input + output, family = binomial, 
    data = scaled_linfwd_gpt35_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.93263  -0.26941  -0.04796  -0.01200   2.05953  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -0.1080     0.3119  -0.346 0.729269    
taskconversion_fake  -6.9788     1.8301  -3.813 0.000137 ***
input                -0.8874     0.9413  -0.943 0.345840    
output               -0.8883     0.8660  -1.026 0.305031    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 218.099  on 199  degrees of freedom
Residual deviance:  88.084  on 196  degrees of freedom
AIC: 96.084

Number of Fisher Scoring iterations: 16


In [225]:
summary(linrev_gpt4_model)


Call:
bayesglm(formula = correct ~ task + input + output, family = binomial, 
    data = scaled_linrev_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.40818  -0.60031  -0.07781  -0.02974   1.97667  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)                 -0.3915     0.2716  -1.442 0.149357    
taskconversion_fakeinverse  -5.6718     1.6886  -3.359 0.000782 ***
input                       -0.4566     0.8349  -0.547 0.584435    
output                      -0.4366     0.7182  -0.608 0.543210    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 210.76  on 199  degrees of freedom
Residual deviance: 122.78  on 196  degrees of freedom
AIC: 130.78

Number of Fisher Scoring iterations: 18


In [226]:
summary(linrev_gpt35_model)


Call:
bayesglm(formula = correct ~ task + input + output, family = binomial, 
    data = scaled_linrev_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8763  -0.2377  -0.1307   0.7056   3.2600  

Coefficients:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)                  0.4090     0.2653   1.541    0.123    
taskconversion_fakeinverse  -4.9606     0.9044  -5.485 4.13e-08 ***
input                       -0.1544     0.8135  -0.190    0.849    
output                      -0.6415     0.7081  -0.906    0.365    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 246.02  on 199  degrees of freedom
Residual deviance: 131.42  on 196  degrees of freedom
AIC: 139.42

Number of Fisher Scoring iterations: 11


In [223]:
vif(linfwd_gpt4_model)

task    input   output 
1.012380 9.033595 9.070444

In [224]:
vif(linfwd_gpt35_model)

task    input   output 
1.029425 7.692754 7.751149

In [227]:
vif(linrev_gpt4_model)

task    input   output 
1.010568 9.608806 9.636934

In [228]:
vif(linrev_gpt35_model)

task     input    output 
 1.061963 10.628090 10.815075

### Comparing methods

In [229]:
linmethod_gpt4_df <- read.table(file = 'table_conversion_method_gpt-4-0613.tsv', sep = '\t', header = TRUE)
linmethod_gpt35_df <- read.table(file = 'table_conversion_method_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [230]:
scaled_linmethod_gpt4_df <- scale_lin_df(linmethod_gpt4_df)
scaled_linmethod_gpt35_df <- scale_lin_df(linmethod_gpt35_df)

In [231]:
contrasts(scaled_linmethod_gpt4_df$task) <- contr.sum(3)
contrasts(scaled_linmethod_gpt35_df$task) <- contr.sum(3)

In [232]:
linmethod_gpt4_model <- glmer(correct ~ task + (1|index), 
               data=scaled_linmethod_gpt4_df, family=binomial)
linmethod_gpt35_model <- glmer(correct ~ task + (1|index), 
               data=scaled_linmethod_gpt35_df , family=binomial)

In [233]:
linmethod_gpt4_null_model <- glmer(correct ~ (1|index), 
               data=scaled_linmethod_gpt4_df, family=binomial)
linmethod_gpt35_null_model <- glmer(correct ~ (1|index), 
               data=scaled_linmethod_gpt35_df , family=binomial)

In [234]:
summary(linmethod_gpt4_model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: correct ~ task + (1 | index)
   Data: scaled_linmethod_gpt4_df

     AIC      BIC   logLik deviance df.resid 
   311.7    326.5   -151.9    303.7      296 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.25915 -0.20466  0.04086  0.44265  2.02756 

Random effects:
 Groups Name        Variance Std.Dev.
 index  (Intercept) 11.22    3.35    
Number of obs: 300, groups:  index, 100

Fixed effects:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   0.9127     0.4387   2.081   0.0375 *  
task1        -2.6689     0.5598  -4.767 1.87e-06 ***
task2         2.5676     0.5447   4.713 2.44e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
      (Intr) task1 
task1 -0.354       
task2  0.350 -0.870

In [235]:
summary(linmethod_gpt35_model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: correct ~ task + (1 | index)
   Data: scaled_linmethod_gpt35_df

     AIC      BIC   logLik deviance df.resid 
   322.8    337.6   -157.4    314.8      296 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.2141 -0.3941  0.2185  0.4516  2.5372 

Random effects:
 Groups Name        Variance Std.Dev.
 index  (Intercept) 5.497    2.345   
Number of obs: 300, groups:  index, 100

Fixed effects:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   1.3924     0.3640   3.826  0.00013 ***
task1        -1.5561     0.3001  -5.185 2.16e-07 ***
task2         1.8958     0.3556   5.331 9.74e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
      (Intr) task1 
task1 -0.374       
task2  0.414 -0.735

In [236]:
anova(linmethod_gpt4_model,linmethod_gpt4_null_model,test="Chisq")

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
linmethod_gpt4_null_model,2,392.7577,400.1652,-194.3788,388.7577,NA,NA,NA
linmethod_gpt4_model,4,311.7095,326.5246,-151.8547,303.7095,85.04822,2,3.404193e-19


In [237]:
anova(linmethod_gpt35_model,linmethod_gpt35_null_model,test="Chisq")

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
linmethod_gpt35_null_model,2,374.2759,381.6835,-185.1379,370.2759,NA,NA,NA
linmethod_gpt35_model,4,322.8321,337.6472,-157.4161,314.8321,55.44378,2,9.131353e-13


In [238]:
gpt4_lin_multcomp <- glht(linmethod_gpt4_model, linfct=mcp(task="Tukey"))
gpt35_lin_multcomp <- glht(linmethod_gpt35_model, linfct=mcp(task="Tukey"))


In [239]:
summary(gpt4_lin_multcomp)


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: glmer(formula = correct ~ task + (1 | index), data = scaled_linmethod_gpt4_df, 
    family = binomial)

Linear Hypotheses:
                                                              Estimate
conversion_actualprimed - conversion_actual == 0                5.2365
conversion_actualprimedcontrol - conversion_actual == 0         2.7703
conversion_actualprimedcontrol - conversion_actualprimed == 0  -2.4662
                                                              Std. Error
conversion_actualprimed - conversion_actual == 0                  1.0681
conversion_actualprimedcontrol - conversion_actual == 0           0.6992
conversion_actualprimedcontrol - conversion_actualprimed == 0     0.6625
                                                              z value Pr(>|z|)
conversion_actualprimed - conversion_actual == 0                4.903  < 1e-04
conversion_actualprimedcontrol - con

In [240]:
summary(gpt35_lin_multcomp)


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: glmer(formula = correct ~ task + (1 | index), data = scaled_linmethod_gpt35_df, 
    family = binomial)

Linear Hypotheses:
                                                              Estimate
conversion_actualprimed - conversion_actual == 0                3.4519
conversion_actualprimedcontrol - conversion_actual == 0         1.2163
conversion_actualprimedcontrol - conversion_actualprimed == 0  -2.2356
                                                              Std. Error
conversion_actualprimed - conversion_actual == 0                  0.6110
conversion_actualprimedcontrol - conversion_actual == 0           0.4161
conversion_actualprimedcontrol - conversion_actualprimed == 0     0.5313
                                                              z value Pr(>|z|)
conversion_actualprimed - conversion_actual == 0                5.650  < 0.001
conversion_actualprimedcontrol - co

### Basic test: OOD

In [241]:
# Read in data
linfwdood_gpt4_df <- read.table(file = 'table_conversion_ood_fwd_gpt-4-0613.tsv', sep = '\t', header = TRUE)
linfwdood_gpt35_df <- read.table(file = 'table_conversion_ood_fwd_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

linrevood_gpt4_df <- read.table(file = 'table_conversion_ood_rev_gpt-4-0613.tsv', sep = '\t', header = TRUE)
linrevood_gpt35_df <- read.table(file = 'table_conversion_ood_rev_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

In [242]:
scaled_linfwdood_gpt4_df <- scale_lin_df(linfwdood_gpt4_df)
scaled_linfwdood_gpt35_df <- scale_lin_df(linfwdood_gpt35_df)
scaled_linrevood_gpt4_df <- scale_lin_df(linrevood_gpt4_df)
scaled_linrevood_gpt35_df <- scale_lin_df(linrevood_gpt35_df)

In [243]:
linfwdood_gpt4_model <- bayesglm(correct ~ task + input + output, 
               data=scaled_linfwdood_gpt4_df, family=binomial)
linfwdood_gpt35_model <- bayesglm(correct ~ task + input + output, 
               data=scaled_linfwdood_gpt35_df, family=binomial)
linrevood_gpt4_model <- bayesglm(correct ~ task + input + output, 
               data=scaled_linrevood_gpt4_df, family=binomial)
linrevood_gpt35_model <- bayesglm(correct ~ task + input + output, 
               data=scaled_linrevood_gpt35_df, family=binomial)

In [244]:
summary(linfwdood_gpt4_model)


Call:
bayesglm(formula = correct ~ task + input + output, family = binomial, 
    data = scaled_linfwdood_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.7808  -0.3932  -0.1228  -0.0655   2.4532  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)              -2.2336     0.5434  -4.110 3.95e-05 ***
taskconversion_ood_fake  -3.6606     1.6255  -2.252   0.0243 *  
input                    -0.5776     0.7349  -0.786   0.4319    
output                   -0.1299     0.7855  -0.165   0.8687    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 85.193  on 199  degrees of freedom
Residual deviance: 64.937  on 196  degrees of freedom
AIC: 72.937

Number of Fisher Scoring iterations: 21


In [245]:
summary(linfwdood_gpt35_model)


Call:
bayesglm(formula = correct ~ task + input + output, family = binomial, 
    data = scaled_linfwdood_gpt35_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.41999  -0.33987  -0.20057  -0.06683   2.69279  

Coefficients:
                        Estimate Std. Error z value Pr(>|z|)   
(Intercept)              -1.8413     0.5660  -3.253  0.00114 **
taskconversion_ood_fake  -3.3915     1.1394  -2.977  0.00292 **
input                    -1.1050     0.7961  -1.388  0.16512   
output                   -0.3337     0.8472  -0.394  0.69364   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 130.033  on 199  degrees of freedom
Residual deviance:  83.716  on 196  degrees of freedom
AIC: 91.716

Number of Fisher Scoring iterations: 15


In [246]:
summary(linrevood_gpt4_model)


Call:
bayesglm(formula = correct ~ task + input + output, family = binomial, 
    data = scaled_linrevood_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.85211  -0.69168  -0.09061  -0.07368   1.85504  

Coefficients:
                               Estimate Std. Error z value Pr(>|z|)   
(Intercept)                     -1.2347     0.4630  -2.666  0.00767 **
taskconversion_ood_fakeinverse  -4.4110     1.7146  -2.573  0.01009 * 
input                           -0.1056     0.8171  -0.129  0.89714   
output                           0.3170     0.6315   0.502  0.61571   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 146.77  on 199  degrees of freedom
Residual deviance: 109.74  on 196  degrees of freedom
AIC: 117.74

Number of Fisher Scoring iterations: 20


In [247]:
summary(linrevood_gpt35_model)


Call:
bayesglm(formula = correct ~ task + input + output, family = binomial, 
    data = scaled_linrevood_gpt35_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.05965  -0.83455  -0.08504  -0.06645   1.60478  

Coefficients:
                                Estimate Std. Error z value Pr(>|z|)   
(Intercept)                    -0.792035   0.458688  -1.727  0.08421 . 
taskconversion_ood_fakeinverse -4.812195   1.725003  -2.790  0.00528 **
input                          -0.310161   0.823762  -0.377  0.70653   
output                          0.003018   0.631042   0.005  0.99618   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 185.49  on 199  degrees of freedom
Residual deviance: 128.92  on 196  degrees of freedom
AIC: 136.92

Number of Fisher Scoring iterations: 19


In [248]:
vif(linfwdood_gpt4_model)

task    input   output 
1.175392 4.827758 5.031327

In [249]:
vif(linfwdood_gpt35_model)

task    input   output 
1.707186 5.409668 6.500893

In [250]:
vif(linrevood_gpt4_model)

task    input   output 
1.201113 7.664596 7.441326

In [251]:
vif(linrevood_gpt35_model)

task    input   output 
1.158872 9.465915 9.362273

### Comparing methods: OOD

In [252]:
linmethodood_gpt4_df <- read.table(file = 'table_conversion_ood_method_gpt-4-0613.tsv', sep = '\t', header = TRUE)
linmethodood_gpt35_df <- read.table(file = 'table_conversion_ood_method_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)


In [253]:
scaled_linmethodood_gpt4_df <- scale_lin_df(linmethodood_gpt4_df)
scaled_linmethodood_gpt35_df <- scale_lin_df(linmethodood_gpt35_df)


In [254]:
contrasts(scaled_linmethodood_gpt4_df$task) <- contr.sum(3)
contrasts(scaled_linmethodood_gpt35_df$task) <- contr.sum(3)

In [255]:
linmethodood_gpt4_model <- glmer(correct ~ task + (1|index), 
               data=scaled_linmethodood_gpt4_df, family=binomial, control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))
linmethodood_gpt35_model <- glmer(correct ~ task + (1|index), 
               data=scaled_linmethodood_gpt35_df , family=binomial, control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))


In [256]:
linmethodood_gpt4_null_model <- glmer(correct ~ (1|index), 
               data=scaled_linmethodood_gpt4_df, family=binomial, control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))
linmethodood_gpt35_null_model <- glmer(correct ~ (1|index), 
               data=scaled_linmethodood_gpt35_df , family=binomial, control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5)))


In [257]:
summary(linmethodood_gpt4_model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: correct ~ task + (1 | index)
   Data: scaled_linmethodood_gpt4_df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
   293.2    308.0   -142.6    285.2      296 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9160 -0.2553 -0.0708  0.1482  3.9168 

Random effects:
 Groups Name        Variance Std.Dev.
 index  (Intercept) 14.51    3.809   
Number of obs: 300, groups:  index, 100

Fixed effects:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -1.7345     0.5903  -2.939   0.0033 ** 
task1        -3.5266     0.8475  -4.161 3.17e-05 ***
task2         3.0222     0.7353   4.110 3.96e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
      (Intr) task1 
task1  0.575       
task2 -0.577 -0.927

In [258]:
summary(linmethodood_gpt35_model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: correct ~ task + (1 | index)
   Data: scaled_linmethodood_gpt35_df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

     AIC      BIC   logLik deviance df.resid 
   309.1    323.9   -150.5    301.1      296 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.9842 -0.4462 -0.0373  0.2255  2.1213 

Random effects:
 Groups Name        Variance Std.Dev.
 index  (Intercept) 12.06    3.472   
Number of obs: 300, groups:  index, 100

Fixed effects:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -1.0616     0.4753  -2.233   0.0255 *  
task1        -2.5275     0.5756  -4.391 1.13e-05 ***
task2         2.8639     0.6580   4.352 1.35e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
      (Intr) task1 
task1  0.431       
task2 -0.447 -0.886

In [259]:
anova(linmethodood_gpt4_model,linmethodood_gpt4_null_model,test="Chisq")

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
linmethodood_gpt4_null_model,2,387.6053,395.0128,-191.8026,383.6053,NA,NA,NA
linmethodood_gpt4_model,4,293.2305,308.0456,-142.6152,285.2305,98.37479,2,4.346951e-22


In [260]:
anova(linmethodood_gpt35_model,linmethodood_gpt35_null_model,test="Chisq")

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
linmethodood_gpt35_null_model,2,391.3457,398.7533,-193.6729,387.3457,NA,NA,NA
linmethodood_gpt35_model,4,309.0817,323.8968,-150.5409,301.0817,86.26404,2,1.853538e-19


In [261]:
gpt4_linood_multcomp <- glht(linmethodood_gpt4_model, linfct=mcp(task="Tukey"))

In [262]:
gpt35_linood_multcomp <- glht(linmethodood_gpt35_model, linfct=mcp(task="Tukey"))

In [263]:
summary(gpt4_linood_multcomp)


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: glmer(formula = correct ~ task + (1 | index), data = scaled_linmethodood_gpt4_df, 
    family = binomial, control = glmerControl(optimizer = "bobyqa", 
        optCtrl = list(maxfun = 2e+05)))

Linear Hypotheses:
                                                                      Estimate
conversion_ood_actualprimed - conversion_ood_actual == 0                6.5488
conversion_ood_actualprimedcontrol - conversion_ood_actual == 0         4.0310
conversion_ood_actualprimedcontrol - conversion_ood_actualprimed == 0  -2.5178
                                                                      Std. Error
conversion_ood_actualprimed - conversion_ood_actual == 0                  1.5538
conversion_ood_actualprimedcontrol - conversion_ood_actual == 0           1.0505
conversion_ood_actualprimedcontrol - conversion_ood_actualprimed == 0     0.7555
                                        

In [264]:
summary(gpt35_linood_multcomp)


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: glmer(formula = correct ~ task + (1 | index), data = scaled_linmethodood_gpt35_df, 
    family = binomial, control = glmerControl(optimizer = "bobyqa", 
        optCtrl = list(maxfun = 2e+05)))

Linear Hypotheses:
                                                                      Estimate
conversion_ood_actualprimed - conversion_ood_actual == 0                5.3914
conversion_ood_actualprimedcontrol - conversion_ood_actual == 0         2.1912
conversion_ood_actualprimedcontrol - conversion_ood_actualprimed == 0  -3.2002
                                                                      Std. Error
conversion_ood_actualprimed - conversion_ood_actual == 0                  1.1982
conversion_ood_actualprimedcontrol - conversion_ood_actual == 0           0.6444
conversion_ood_actualprimedcontrol - conversion_ood_actualprimed == 0     0.8488
                                       

# Counting

In [167]:
# Read in data
countwords_gpt4_df <- read.table(file = 'table_counting_words_gpt-4-0613.tsv', sep = '\t', header = TRUE)
countwords_gpt35_df <- read.table(file = 'table_counting_words_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

countchars_gpt4_df <- read.table(file = 'table_counting_chars_gpt-4-0613.tsv', sep = '\t', header = TRUE)
countchars_gpt35_df <- read.table(file = 'table_counting_chars_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)


In [168]:
# Z-score data
scaled_countwords_gpt4_df <- scale_df_with_index(countwords_gpt4_df)
scaled_countwords_gpt35_df <- scale_df_with_index(countwords_gpt35_df)

scaled_countchars_gpt4_df <- scale_df_with_index(countchars_gpt4_df)
scaled_countchars_gpt35_df <- scale_df_with_index(countchars_gpt35_df)

In [169]:
head(scaled_countwords_gpt4_df)

,index,input_nchars,input_ntokens,input_logprob,output_nchars,output_ntokens,output_logprob,correct
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,-1.71453,-1.741827,-1.739786,-4.4424536,-3.006613,NaN,2.316207,1
2,-1.71453,-1.738059,-1.771344,8.7549067,-3.006613,NaN,2.316207,1
3,-1.71453,-1.726757,-1.755565,-1.9608672,-3.006613,NaN,2.316207,1
4,-1.71453,-1.738059,-1.739786,-4.6237970,-3.006613,NaN,2.316207,1
5,-1.71453,-1.726757,-1.739786,-3.5102910,-3.006613,NaN,2.316207,1
6,-1.71453,-1.726757,-1.739786,-0.5541231,-3.006613,NaN,2.316207,1


In [170]:
countwords_gpt4_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countwords_gpt4_df, family=binomial)
countwords_gpt35_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countwords_gpt35_df, family=binomial)


countchars_gpt4_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countchars_gpt4_df, family=binomial)
countchars_gpt35_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countchars_gpt35_df, family=binomial)


In [171]:
summary(countwords_gpt4_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countwords_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.57541  -0.16382  -0.04986  -0.02723   2.91722  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.35701    0.15836 -21.199   <2e-16 ***
index          -0.19417    0.10797  -1.798   0.0721 .  
input_logprob  -0.10037    0.08225  -1.220   0.2224    
output_logprob  3.59077    0.17453  20.574   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3264.4  on 2999  degrees of freedom
Residual deviance: 1137.6  on 2996  degrees of freedom
AIC: 1145.6

Number of Fisher Scoring iterations: 7


In [172]:
summary(countwords_gpt35_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countwords_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.0485  -0.2589  -0.0675  -0.0346   3.4371  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -4.31424    0.19524 -22.098   <2e-16 ***
index           0.06015    0.10323   0.583    0.560    
input_logprob  -0.01359    0.05816  -0.234    0.815    
output_logprob  2.95668    0.15685  18.851   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2283.9  on 2999  degrees of freedom
Residual deviance: 1074.0  on 2996  degrees of freedom
AIC: 1082

Number of Fisher Scoring iterations: 7


In [173]:
summary(countchars_gpt4_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countchars_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.46780  -0.15937  -0.03976   0.07145   3.08559  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.08292    0.15416 -19.998  < 2e-16 ***
index          -0.82241    0.10574  -7.778  7.4e-15 ***
input_logprob  -0.05048    0.07837  -0.644     0.52    
output_logprob  3.55316    0.17447  20.365  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3505.5  on 2999  degrees of freedom
Residual deviance: 1070.3  on 2996  degrees of freedom
AIC: 1078.3

Number of Fisher Scoring iterations: 7


In [174]:
summary(countchars_gpt35_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countchars_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.0301  -0.5265  -0.2664   0.4923   2.3786  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -1.08394    0.05786 -18.733   <2e-16 ***
index          -0.71066    0.07709  -9.218   <2e-16 ***
input_logprob  -0.06039    0.05344  -1.130    0.258    
output_logprob  1.44736    0.08107  17.852   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3836.9  on 2999  degrees of freedom
Residual deviance: 2254.9  on 2996  degrees of freedom
AIC: 2262.9

Number of Fisher Scoring iterations: 5


In [175]:
vif(countwords_gpt4_model)

index  input_logprob output_logprob 
      1.162683       1.040762       1.119767

In [176]:
vif(countwords_gpt35_model)

index  input_logprob output_logprob 
      1.211863       1.032591       1.177339

In [177]:
vif(countchars_gpt4_model)

index  input_logprob output_logprob 
      1.017713       1.010703       1.011115

In [178]:
vif(countchars_gpt35_model)

index  input_logprob output_logprob 
      1.447074       1.005327       1.441650

### Varying input

In [190]:
# Read in data
countwords_both_gpt4_df <- read.table(file = 'table_counting_words_both_gpt-4-0613.tsv', sep = '\t', header = TRUE)
countwords_both_gpt35_df <- read.table(file = 'table_counting_words_both_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

countchars_both_gpt4_df <- read.table(file = 'table_counting_chars_both_gpt-4-0613.tsv', sep = '\t', header = TRUE)
countchars_both_gpt35_df <- read.table(file = 'table_counting_chars_both_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)


In [191]:
# Z-score data
scaled_countwords_both_gpt4_df <- scale_df_with_index(countwords_both_gpt4_df)
scaled_countwords_both_gpt35_df <- scale_df_with_index(countwords_both_gpt35_df)

scaled_countchars_both_gpt4_df <- scale_df_with_index(countchars_both_gpt4_df)
scaled_countchars_both_gpt35_df <- scale_df_with_index(countchars_both_gpt35_df)

In [192]:
countwords_both_gpt4_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countwords_both_gpt4_df, family=binomial)
countwords_both_gpt35_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countwords_both_gpt35_df, family=binomial)

countchars_both_gpt4_model <- glm(correct ~ index + input_logprob + output_logprob + input_ntokens, 
                   data=scaled_countchars_both_gpt4_df, family=binomial)
countchars_both_gpt35_model <- glm(correct ~ index + input_logprob + output_logprob + input_ntokens, 
                   data=scaled_countchars_both_gpt35_df, family=binomial)

In [193]:
summary(countwords_both_gpt4_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countwords_both_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.85516  -0.14205  -0.04240  -0.02356   3.10665  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.597483   0.121388 -29.636   <2e-16 ***
index           0.006453   0.079863   0.081   0.9356    
input_logprob   0.117880   0.054903   2.147   0.0318 *  
output_logprob  3.927246   0.138109  28.436   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6427.5  on 5999  degrees of freedom
Residual deviance: 2162.5  on 5996  degrees of freedom
AIC: 2170.5

Number of Fisher Scoring iterations: 7


In [194]:
summary(countwords_both_gpt35_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countwords_both_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8573  -0.3448  -0.1093  -0.0759   3.2168  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.47416    0.09882 -35.157  < 2e-16 ***
index           0.43608    0.07043   6.192 5.95e-10 ***
input_logprob  -0.11203    0.05037  -2.224   0.0261 *  
output_logprob  2.63160    0.09325  28.222  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 4749.4  on 5999  degrees of freedom
Residual deviance: 2581.4  on 5996  degrees of freedom
AIC: 2589.4

Number of Fisher Scoring iterations: 7


In [195]:
summary(countchars_both_gpt4_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob + 
    input_ntokens, family = binomial, data = scaled_countchars_both_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.9362  -0.1098  -0.0262   0.0312   3.4657  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -4.1767     0.1621 -25.768  < 2e-16 ***
index           -0.6219     0.1174  -5.296 1.18e-07 ***
input_logprob   -0.6803     0.1281  -5.313 1.08e-07 ***
output_logprob   3.3781     0.1396  24.202  < 2e-16 ***
input_ntokens   -2.5456     0.3061  -8.315  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6819.6  on 5999  degrees of freedom
Residual deviance: 1846.0  on 5995  degrees of freedom
AIC: 1856

Number of Fisher Scoring iterations: 8


In [196]:
summary(countchars_both_gpt35_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob + 
    input_ntokens, family = binomial, data = scaled_countchars_both_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2093  -0.3962  -0.1922   0.2725   2.6262  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -1.86072    0.05794 -32.115  < 2e-16 ***
index          -0.38644    0.08268  -4.674 2.95e-06 ***
input_logprob   0.49788    0.07428   6.702 2.05e-11 ***
output_logprob  2.01289    0.07029  28.636  < 2e-16 ***
input_ntokens  -0.24959    0.12127  -2.058   0.0396 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 7098.4  on 5999  degrees of freedom
Residual deviance: 3590.7  on 5995  degrees of freedom
AIC: 3600.7

Number of Fisher Scoring iterations: 6


In [197]:
vif(countwords_both_gpt4_model)

index  input_logprob output_logprob 
      1.192022       1.005355       1.195770

In [198]:
vif(countwords_both_gpt35_model)

index  input_logprob output_logprob 
      1.523886       1.002745       1.526534

In [199]:
vif(countchars_both_gpt4_model)

index  input_logprob output_logprob  input_ntokens 
      1.540032       4.394968       1.021099       4.345042

In [200]:
vif(countchars_both_gpt35_model)

index  input_logprob output_logprob  input_ntokens 
      2.622199       2.981228       1.480175       3.401534

### Varying output

In [179]:
# Read in data
countwords_binary_gpt4_df <- read.table(file = 'table_counting_words_binary_gpt-4-0613.tsv', sep = '\t', header = TRUE)
countwords_binary_gpt35_df <- read.table(file = 'table_counting_words_binary_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

countchars_binary_gpt4_df <- read.table(file = 'table_counting_chars_binary_gpt-4-0613.tsv', sep = '\t', header = TRUE)
countchars_binary_gpt35_df <- read.table(file = 'table_counting_chars_binary_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)


In [180]:
# Z-score data
scaled_countwords_binary_gpt4_df <- scale_df_with_index(countwords_binary_gpt4_df)
scaled_countwords_binary_gpt35_df <- scale_df_with_index(countwords_binary_gpt35_df)

scaled_countchars_binary_gpt4_df <- scale_df_with_index(countchars_binary_gpt4_df)
scaled_countchars_binary_gpt35_df <- scale_df_with_index(countchars_binary_gpt35_df)

In [181]:
countwords_binary_gpt4_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countwords_binary_gpt4_df, family=binomial)
countwords_binary_gpt35_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countwords_binary_gpt35_df, family=binomial)

countchars_binary_gpt4_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countchars_binary_gpt4_df, family=binomial)
countchars_binary_gpt35_model <- glm(correct ~ index + input_logprob + output_logprob, 
                   data=scaled_countchars_binary_gpt35_df, family=binomial)

In [182]:
summary(countwords_binary_gpt4_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countwords_binary_gpt4_df)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-2.13331  -0.37089  -0.08181  -0.04723   2.96403  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -2.73270    0.12591 -21.704  < 2e-16 ***
index           0.52051    0.08623   6.036 1.58e-09 ***
input_logprob   0.12293    0.06573   1.870   0.0615 .  
output_logprob  3.22597    0.15411  20.933  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2633.6  on 2399  degrees of freedom
Residual deviance: 1259.9  on 2396  degrees of freedom
AIC: 1267.9

Number of Fisher Scoring iterations: 7


In [183]:
summary(countwords_binary_gpt35_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countwords_binary_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6350  -0.3409  -0.1558  -0.0843   3.4431  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     -3.8385     0.1616 -23.747   <2e-16 ***
index            1.0924     0.0941  11.609   <2e-16 ***
input_logprob   -0.2086     0.1051  -1.984   0.0473 *  
output_logprob   1.8626     0.1338  13.917   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1154.44  on 2399  degrees of freedom
Residual deviance:  764.18  on 2396  degrees of freedom
AIC: 772.18

Number of Fisher Scoring iterations: 7


In [184]:
summary(countchars_binary_gpt4_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countchars_binary_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8510  -0.4543  -0.1344  -0.0681   2.2621  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -2.65994    0.11628 -22.875   <2e-16 ***
index           0.01801    0.06977   0.258    0.796    
input_logprob  -0.09741    0.06844  -1.423    0.155    
output_logprob  2.54242    0.11741  21.655   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2488.0  on 2399  degrees of freedom
Residual deviance: 1364.7  on 2396  degrees of freedom
AIC: 1372.7

Number of Fisher Scoring iterations: 6


In [185]:
summary(countchars_binary_gpt35_model)


Call:
glm(formula = correct ~ index + input_logprob + output_logprob, 
    family = binomial, data = scaled_countchars_binary_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.8611  -0.6616  -0.2926   0.5626   2.4637  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -1.48069    0.06776 -21.851   <2e-16 ***
index          -0.53780    0.05743  -9.365   <2e-16 ***
input_logprob  -0.03054    0.05630  -0.542    0.588    
output_logprob  1.37065    0.07117  19.259   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 2821.3  on 2399  degrees of freedom
Residual deviance: 1974.3  on 2396  degrees of freedom
AIC: 1982.3

Number of Fisher Scoring iterations: 5


In [186]:
vif(countwords_binary_gpt4_model)

index  input_logprob output_logprob 
      1.384991       1.003427       1.388899

In [187]:
vif(countwords_binary_gpt35_model)

index  input_logprob output_logprob 
      1.102513       1.026916       1.075096

In [188]:
vif(countchars_binary_gpt4_model)

index  input_logprob output_logprob 
      1.065096       1.005404       1.070308

In [189]:
vif(countchars_binary_gpt35_model)

index  input_logprob output_logprob 
      1.019215       1.001138       1.020041

# Sorting

In [201]:
# Read in data
sortwords_gpt4_df <- read.table(file = 'table_sortwords_gpt-4-0613.tsv', sep = '\t', header = TRUE)
sortwords_gpt35_df <- read.table(file = 'table_sortwords_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)

sortnumbers_gpt4_df <- read.table(file = 'table_sortnumbers_gpt-4-0613.tsv', sep = '\t', header = TRUE)
sortnumbers_gpt35_df <- read.table(file = 'table_sortnumbers_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)


In [202]:
# Z-score data
scaled_sortwords_gpt4_df <- scale_taskpair_df(sortwords_gpt4_df)
scaled_sortwords_gpt35_df <- scale_taskpair_df(sortwords_gpt35_df)

scaled_sortnumbers_gpt4_df <- scale_taskpair_df(sortnumbers_gpt4_df)
scaled_sortnumbers_gpt35_df <- scale_taskpair_df(sortnumbers_gpt35_df)

In [203]:
head(scaled_sortwords_gpt4_df)

,input_nchars,input_ntokens,input_logprob,output_nchars,output_ntokens,output_logprob,index,task,correct
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<int>
1,-1.4316216,-1.0266752,0.9946478,-1.4316216,-1.0266752,1.1707491,0,fwd,1
2,-0.5566579,-0.6333130,0.8375733,-0.5566579,-0.6333130,1.0567959,1,fwd,1
3,0.4389904,0.0550707,-0.4429823,0.4389904,0.0550707,-0.4758583,2,fwd,1
4,-1.1902523,-1.1250157,0.7908320,-1.1902523,-1.1250157,1.0975093,3,fwd,1
5,-1.1902523,-1.1250157,1.3138576,-1.1902523,-1.1250157,1.0760712,4,fwd,1
6,1.4949810,1.3334976,-1.6865322,1.4949810,1.3334976,-1.3011868,5,fwd,1


In [204]:
sort_gpt4_words_model <- glm(correct ~ task + input_nchars + input_ntokens + input_logprob + output_logprob, 
               data=scaled_sortwords_gpt4_df, family=binomial)
sort_gpt35_words_model <- glm(correct ~ task + input_nchars + input_ntokens + input_logprob + output_logprob, 
               data=scaled_sortwords_gpt35_df, family=binomial)


In [205]:
sort_gpt4_numbers_model <- glm(correct ~ task + input_nchars + input_ntokens + input_logprob + output_logprob, 
               data=scaled_sortnumbers_gpt4_df, family=binomial)
sort_gpt35_numbers_model <- glm(correct ~ task + input_nchars + input_ntokens + input_logprob + output_logprob, 
               data=scaled_sortnumbers_gpt35_df, family=binomial)

In [206]:
summary(sort_gpt4_words_model)


Call:
glm(formula = correct ~ task + input_nchars + input_ntokens + 
    input_logprob + output_logprob, family = binomial, data = scaled_sortwords_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2414  -0.9179   0.4780   0.7151   2.0277  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     1.50008    0.31078   4.827 1.39e-06 ***
taskrev        -2.32891    0.45518  -5.116 3.11e-07 ***
input_nchars    0.65920    0.64311   1.025    0.305    
input_ntokens  -0.99387    0.87178  -1.140    0.254    
input_logprob   0.25661    0.90998   0.282    0.778    
output_logprob -0.02024    0.78071  -0.026    0.979    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 274.37  on 199  degrees of freedom
Residual deviance: 212.31  on 194  degrees of freedom
AIC: 224.31

Number of Fisher Scoring iterations: 4


In [207]:
summary(sort_gpt35_words_model)


Call:
glm(formula = correct ~ task + input_nchars + input_ntokens + 
    input_logprob + output_logprob, family = binomial, data = scaled_sortwords_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9744  -0.6160  -0.3827   0.7223   2.2874  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.0275     0.2797   3.674 0.000239 ***
taskrev         -2.6897     0.4560  -5.898 3.67e-09 ***
input_nchars    -0.2244     0.7079  -0.317 0.751250    
input_ntokens    0.2777     0.9500   0.292 0.770045    
input_logprob   -0.4364     0.9579  -0.456 0.648674    
output_logprob   0.9603     0.8631   1.113 0.265902    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 275.64  on 199  degrees of freedom
Residual deviance: 188.36  on 194  degrees of freedom
AIC: 200.36

Number of Fisher Scoring iterations: 4


In [208]:
summary(sort_gpt4_numbers_model)


Call:
glm(formula = correct ~ task + input_nchars + input_ntokens + 
    input_logprob + output_logprob, family = binomial, data = scaled_sortnumbers_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2265   0.4663   0.5864   0.6988   0.9245  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     1.52436    0.35167   4.335 1.46e-05 ***
taskdescending -0.03131    0.58923  -0.053    0.958    
input_nchars   -8.09348    6.40191  -1.264    0.206    
input_ntokens   6.68226    6.03090   1.108    0.268    
input_logprob  -1.26602    1.20055  -1.055    0.292    
output_logprob  0.19837    0.89414   0.222    0.824    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 194.49  on 199  degrees of freedom
Residual deviance: 188.87  on 194  degrees of freedom
AIC: 200.87

Number of Fisher Scoring iterations: 4


In [209]:
summary(sort_gpt35_numbers_model)


Call:
glm(formula = correct ~ task + input_nchars + input_ntokens + 
    input_logprob + output_logprob, family = binomial, data = scaled_sortnumbers_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.9000  -1.0069   0.6580   0.8908   2.0472  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      0.6417     0.3041   2.110   0.0349 *  
taskdescending  -0.1977     0.5088  -0.389   0.6976    
input_nchars   -23.8710     6.0243  -3.962 7.42e-05 ***
input_ntokens   25.0612     5.7643   4.348 1.38e-05 ***
input_logprob    1.0765     1.0595   1.016   0.3096    
output_logprob   0.3905     0.7771   0.503   0.6153    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 265.63  on 199  degrees of freedom
Residual deviance: 234.39  on 194  degrees of freedom
AIC: 246.39

Number of Fisher Scoring iterations: 4


In [210]:
vif(sort_gpt4_words_model)

task   input_nchars  input_ntokens  input_logprob output_logprob 
      1.783026      14.384183      26.238785      28.600381      19.170788

In [211]:
vif(sort_gpt35_words_model)

task   input_nchars  input_ntokens  input_logprob output_logprob 
      1.511017      14.802255      26.551489      27.117076      19.080542

In [212]:
vif(sort_gpt4_numbers_model)

task   input_nchars  input_ntokens  input_logprob output_logprob 
      2.592363    1125.507789    1000.267966      40.284436      22.516380

In [213]:
vif(sort_gpt35_numbers_model)

task   input_nchars  input_ntokens  input_logprob output_logprob 
      2.580781    1429.294244    1310.739183      44.169531      23.474987

# Birthdays

In [278]:
# Read in data
birthdays_gpt4_df <- read.table(file = 'table_birthdays_gpt-4-0613.tsv', sep = '\t', header = TRUE)
birthdays_gpt35_df <- read.table(file = 'table_birthdays_gpt-3.5-turbo-0613.tsv', sep = '\t', header = TRUE)


In [279]:
# Z-score data
scaled_birthdays_gpt4_df <- scale_df(birthdays_gpt4_df)
scaled_birthdays_gpt35_df <- scale_df(birthdays_gpt35_df)

In [280]:
birthdays_gpt4_model <- glm(correct ~ input_logprob + output_logprob, 
               data=scaled_birthdays_gpt4_df, family=binomial)
birthdays_gpt35_model <- glm(correct ~ input_logprob + output_logprob, 
               data=scaled_birthdays_gpt35_df, family=binomial)

In [281]:
summary(birthdays_gpt4_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob, family = binomial, 
    data = scaled_birthdays_gpt4_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8769  -0.6683   0.1843   0.5264   2.0459  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.4605     0.1915   7.626 2.43e-14 ***
input_logprob    2.2209     0.2229   9.964  < 2e-16 ***
output_logprob   0.1372     0.1440   0.953    0.341    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 505.92  on 399  degrees of freedom
Residual deviance: 305.07  on 397  degrees of freedom
AIC: 311.07

Number of Fisher Scoring iterations: 6


In [282]:
summary(birthdays_gpt35_model)


Call:
glm(formula = correct ~ input_logprob + output_logprob, family = binomial, 
    data = scaled_birthdays_gpt35_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.9662  -0.4500   0.1340   0.4589   2.3463  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)    0.880415   0.178538   4.931 8.17e-07 ***
input_logprob  2.770293   0.258676  10.710  < 2e-16 ***
output_logprob 0.001702   0.156567   0.011    0.991    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 544.23  on 399  degrees of freedom
Residual deviance: 261.61  on 397  degrees of freedom
AIC: 267.61

Number of Fisher Scoring iterations: 6


In [283]:
vif(birthdays_gpt4_model)

input_logprob output_logprob 
      1.007143       1.007143

In [284]:
vif(birthdays_gpt35_model)

input_logprob output_logprob 
       1.00009        1.00009